#Install Libs

In [ ]:
!pip install torchinfo rasterio scipy scikit-learn matplotlib normflows

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import argparse
import json
import math
import numpy as np
import pathlib
import random
import rasterio
import scipy.io as scio
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import normflows as nf
from torch.autograd import Variable
from torch.distributions import Normal, Bernoulli, LogNormal
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

#Main (Disaster Data)

In [ ]:
# =========================
# Data Loading
# =========================

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.distributions import Normal, Bernoulli

def check_nan(tensor, name):
    if torch.isnan(tensor).any():
        print(f"NaN detected in {name}")
        return True
    return False


import numpy as np

def pruning(BD, LS, LF, sigma, side):
    model = np.zeros(BD.shape)
    if side == "single":
        # 1 - LS alone
        model = model + ((BD == 0) & (LF <= sigma))
        # 2 - LF alone
        model = model + 2 * ((BD == 0) & (LF > sigma) & (LF > LS))
        # 3 - LS and BD
        model = model + 3 * ((BD == 1) & (LF <= sigma))
        # 4 - LF and BD
        model = model + 4 * ((BD == 1) & (LF > sigma) & (LF > LS))
    else:  # double
        # 0 - No hazards (BD=0, LS=0, LF=0)
        model = model + 0 * ((BD < sigma) & (LS < sigma) & (LF < sigma))

        # 1 - LS alone (BD=0, LS>0, LF=0)
        model = model + 1 * ((BD < sigma) & (LS > sigma) & (LF < sigma))

        # 2 - LF alone (BD=0, LF>0, LS=0)
        model = model + 2 * ((BD < sigma) & (LS < sigma) & (LF > sigma))

        # 3 - LS and BD (BD=1, LS>0, LF=0)
        model = model + 3 * ((BD > sigma) & (LS > sigma) & (LF < sigma))

        # 4 - LF and BD (BD=1, LF>0, LS=0)
        model = model + 4 * ((BD > sigma) & (LS < sigma) & (LF > sigma))

        # 5 - LS and LF (BD=0, LS>0, LF>0)
        model = model + 5 * ((BD < sigma) & (LS > sigma) & (LF > sigma))

        # 6 - All hazards (BD=1, LS>0, LF>0)
        model = model + 6 * ((BD > sigma) & (LS > sigma) & (LF > sigma))

    return model


location = '/content/drive/MyDrive/raw data/HaitiEarthquake/PriorsAndObs/'
with rasterio.open(f'{location}DPM.tif') as src:
    Y = src.read(1)
    Y_R = src.transform

with rasterio.open(f'{location}PLS.tif') as src:
    LS = src.read(1)
    LS_R = src.transform

with rasterio.open(f'{location}PLF.tif') as src:
    LF = src.read(1)
    LF_R = src.transform

with rasterio.open(f'{location}PBD.tif') as src:
    BD = src.read(1)
    BD_R = src.transform

with rasterio.open(f'{location}PFT.tif') as src:
    FT = src.read(1)
    FT_R = src.transform

FT[FT > 0] = 1
Y[np.isnan(Y)] = 0
FT[np.isnan(FT)] = 0
LS[np.isnan(LS)] = 0
LF[np.isnan(LF)] = 0
BD[np.isnan(BD)] = 0

# load Geo-Features
location = '/content/drive/MyDrive/raw data/HaitiEarthquake/GeoFeatures/'
with rasterio.open(f'{location}DEM_clipped.tif') as src:
    Geo_DEM = src.read(1)
    Geo_DEM_R = src.transform

with rasterio.open(f'{location}coast_distance_clipped.tif') as src:
    Geo_CD = src.read(1)
    Geo_CD_R = src.transform

with rasterio.open(f'{location}cti_clipped.tif') as src:
    Geo_CTI = src.read(1)
    Geo_CTI_R = src.transform

with rasterio.open(f'{location}land_cover_clipped.tif') as src:
    Geo_LC = src.read(1)
    Geo_LC_R = src.transform

with rasterio.open(f'{location}slope_dem_clipped.tif') as src:
    Geo_Slope = src.read(1)
    Geo_Slope_R = src.transform

with rasterio.open(f'{location}vs30_clipped.tif') as src:
    Geo_VS30 = src.read(1)
    Geo_VS30_R = src.transform

Geo_DEM[np.isnan(Geo_DEM)] = 0
Geo_CD[np.isnan(Geo_CD)] = 0
Geo_CTI[np.isnan(Geo_CTI)] = 0
Geo_LC[np.isnan(Geo_LC)] = 0
Geo_Slope[np.isnan(Geo_Slope)] = 0
Geo_VS30[np.isnan(Geo_VS30)] = 0

def normalize_0_1(data):
    min_val = np.min(data)
    max_val = np.max(data)
    if max_val == min_val:
        return np.zeros_like(data)
    return (data - min_val) / (max_val - min_val)

Geo_DEM = normalize_0_1(Geo_DEM)
Geo_CD = normalize_0_1(Geo_CD)
Geo_CTI = normalize_0_1(Geo_CTI)
Geo_LC = normalize_0_1(Geo_LC)
Geo_Slope = normalize_0_1(Geo_Slope)
Geo_VS30 = normalize_0_1(Geo_VS30)

# Convert Landslide Areal Percentages to Probabilities
new_LS = LS.copy()
index = np.where(LS > 0)
for i in zip(*index):
    p = [4.035, -3.042, 5.237, (-7.592 - np.log(LS[i]))]
    roots = np.roots(p)
    new_LS[i] = np.real(roots[np.isreal(roots)][0])

new_LF = LF.copy()
index = np.where(LF > 0)
for i in zip(*index):
    new_LF[i] = (np.log((np.sqrt(0.4915 / LF[i]) - 1) / 42.40)) / (-9.165)



new_LF[new_LF < 0] = 0
new_LS[new_LS < 0] = 0
new_LS[np.isnan(new_LS)] = 0
new_LF[np.isnan(new_LF)] = 0
P_LF = new_LF.copy()
P_LS = new_LS.copy()

np.save('P_LS.npy', P_LS)
np.save('P_LF.npy', P_LF)

print("Shape of P_LS:", P_LS.shape)
print("Shape of P_LF:", P_LF.shape)

# Classify Local Model by Pruning
prune_type = 'double'
mask = (LS > 0) & (LF > 0)
sigma = 0.15
LOCAL = pruning(FT, P_LS, P_LF, sigma, prune_type)
P_LS[(LOCAL == 5) | (LOCAL == 6)] = np.min(new_LS[new_LS > 0])
P_LF[(LOCAL == 5) | (LOCAL == 6)] = np.min(new_LF[new_LF > 0])


Y_tensor = torch.FloatTensor(Y)
FT_tensor = torch.FloatTensor(FT)
LS_tensor = torch.FloatTensor(P_LS)
LF_tensor = torch.FloatTensor(P_LF)
BD_tensor = torch.FloatTensor(BD)
LOCAL = torch.FloatTensor(LOCAL)

geo_features = [
    Geo_DEM,
    Geo_CD,
    Geo_CTI,
    Geo_LC,
    Geo_Slope,
    Geo_VS30
]

geo_tensors = []
for feature in geo_features:
    feature_tensor = torch.FloatTensor(feature)
    if len(feature_tensor.shape) == 2:
        feature_tensor = feature_tensor.unsqueeze(0)
    geo_tensors.append(feature_tensor)

geo_features_combined = torch.cat(geo_tensors, dim=0)
geo_features_combined = geo_features_combined.permute(1, 2, 0)
print("Main tensors shapes:")
print(f"Y tensor shape: {Y_tensor.shape}")
print(f"FT tensor shape: {FT_tensor.shape}")
print(f"LS tensor shape: {LS_tensor.shape}")
print(f"LF tensor shape: {LF_tensor.shape}")
print(f"BD tensor shape: {BD_tensor.shape}")
print(f"\nCombined geo-features shape: {geo_features_combined.shape}")


PLS = LS_tensor
PLF = LF_tensor
PBD = BD_tensor
GeoFeatures = geo_features_combined
DPM = Y_tensor
BF = FT_tensor
LOCAL = LOCAL

<ipython-input-5-1f3cece55f58>:122: RuntimeWarning: overflow encountered in scalar subtract
  return (data - min_val) / (max_val - min_val)


Shape of P_LS: (2170, 8109)
Shape of P_LF: (2170, 8109)
Main tensors shapes:
Y tensor shape: torch.Size([2170, 8109])
FT tensor shape: torch.Size([2170, 8109])
LS tensor shape: torch.Size([2170, 8109])
LF tensor shape: torch.Size([2170, 8109])
BD tensor shape: torch.Size([2170, 8109])

Combined geo-features shape: torch.Size([2170, 8109, 6])


# Inference

In [ ]:
import sympy

In [ ]:
class Config:
    # Model parameters
    batch_size: int = 1000  # Further reduced
    num_flows: int = 6  # Reduced from 5
    num_inducing: int = 100  # Reduced from 100
    learning_rate: float = 0.001
    epochs: int = 20
    geofeature_dim: int = 6
    hidden_dim: int = 128  # Reduced from 128
    mlp_layers: list = None
    leaky_relu_slope: float = 0.0
    cuda: bool = torch.cuda.is_available()
    device: str = "cuda" if cuda else "cpu"
    log_interval: int = 10
    early_stopping_epochs: int = 15
    anneal_type: str = "std"
    flow_type: str = "planar"
    z_size: int = 1
    max_beta: float = 1.0
    min_beta: float = 0.0
    prune_type: str = "double"
    # New parameters specific to inducing points
    optimize_inducing: bool = True  # Whether to learn inducing point locations
    jitter: float = 1e-5  # Small value added to diagonal of K_ZZ for numerical stability


    # Memory management options
    use_cpu_offload: bool = True  # New parameter
    chunk_size: int = 32  # New parameter
    use_mixed_precision: bool = True  # Use mixed precision for better memory efficiency

    def __post_init__(self):
        self.FloatTensor = torch.cuda.FloatTensor if self.cuda else torch.FloatTensor
        self.mlp_layers = [
            self.geofeature_dim,  # Input dimension - fixed to use geofeature_dim
            self.hidden_dim,     # Hidden layer
            self.hidden_dim,     # Hidden layer
            self.z_size          # Output dimension - changed from batch_size to z_size
        ]

import numpy as np

def pruning(BD, LS, LF, sigma, side):
    model = np.zeros(BD.shape)
    if side == "single":
        # 1 - LS alone
        model = model + ((BD == 0) & (LF <= sigma))
        # 2 - LF alone
        model = model + 2 * ((BD == 0) & (LF > sigma) & (LF > LS))
        # 3 - LS and BD
        model = model + 3 * ((BD == 1) & (LF <= sigma))
        # 4 - LF and BD
        model = model + 4 * ((BD == 1) & (LF > sigma) & (LF > LS))
    else:  # double
        # 0 - No hazards (BD=0, LS=0, LF=0)
        model = model + 0 * ((BD < sigma) & (LS < sigma) & (LF < sigma))

        # 1 - LS alone (BD=0, LS>0, LF=0)
        model = model + 1 * ((BD < sigma) & (LS > sigma) & (LF < sigma))

        # 2 - LF alone (BD=0, LF>0, LS=0)
        model = model + 2 * ((BD < sigma) & (LS < sigma) & (LF > sigma))

        # 3 - LS and BD (BD=1, LS>0, LF=0)
        model = model + 3 * ((BD > sigma) & (LS > sigma) & (LF < sigma))

        # 4 - LF and BD (BD=1, LF>0, LS=0)
        model = model + 4 * ((BD > sigma) & (LS < sigma) & (LF > sigma))

        # 5 - LS and LF (BD=0, LS>0, LF>0)
        model = model + 5 * ((BD < sigma) & (LS > sigma) & (LF > sigma))

        # 6 - All hazards (BD=1, LS>0, LF>0)
        model = model + 6 * ((BD > sigma) & (LS > sigma) & (LF > sigma))

    return model


import torch
import numpy as np
from torch.utils.data import Dataset



class tr_Dataset(Dataset):
    """Dataset class for training the spatial model"""
    def __init__(self, LS, LF, BD, OB, GeoF, FT, LOC):
        self.LS = LS
        self.LF = LF
        self.BD = BD
        self.OB = OB
        self.GeoF = GeoF
        self.FT = FT
        self.LOC = LOC

    def __len__(self):
        return len(self.LS)

    def __getitem__(self, idx):
        return (self.LS[idx],
                self.LF[idx],
                self.BD[idx],
                self.OB[idx],
                self.GeoF[idx],
                self.FT[idx],
                self.LOC[idx])



import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.distributions import Normal, Bernoulli





class Planar(nn.Module):
    """
    Simplified Planar flow that avoids any operations that might cause gradient issues
    """
    def __init__(self, dim=1, scale=0.3, max_shift=3.0):
        super(Planar, self).__init__()
        self.u = nn.Parameter(torch.randn(dim) * scale)
        self.w = nn.Parameter(torch.randn(dim) * scale)
        self.b = nn.Parameter(torch.randn(1) * 0.1 + 0.5)
        self.max_shift = max_shift

    def forward(self, z, *args):
        # Create fresh copies of all tensors
        z_fresh = z.detach().clone().requires_grad_(True)
        u_fresh = self.u.detach().clone().requires_grad_(True)
        w_fresh = self.w.detach().clone().requires_grad_(True)
        b_fresh = self.b.detach().clone().requires_grad_(True)

        # Compute the transformation with all fresh tensors
        w_transposed = w_fresh.unsqueeze(0).transpose(1, 0)
        inner_prod = torch.matmul(z_fresh, w_transposed)
        activated = torch.tanh(inner_prod + b_fresh)

        # Calculate shift with safeguards
        shift_raw = activated * u_fresh.unsqueeze(0)
        shift = torch.clamp(shift_raw, min=-self.max_shift, max=self.max_shift)

        # Create output
        z_next = z_fresh + shift

        # Compute Jacobian determinant
        derivative = 1 - activated * activated
        psi = derivative * w_fresh.unsqueeze(0)
        det_term = 1 + torch.matmul(psi, u_fresh.unsqueeze(0).transpose(1, 0))
        log_det = torch.log(torch.abs(det_term) + 1e-8)

        # Reconnect gradient flow from original inputs to outputs
        z_out = z + (z_next - z_fresh)
        log_det_out = log_det.squeeze()

        return z_out, log_det_out

def matern_kernel(x1, x2, lengthscale=1.0, variance=1.0, nu=1.5):
    """
    Compute Matern kernel between two sets of features.

    Args:
        x1: First set of feature vectors [batch_size, feature_dim]
        x2: Second set of feature vectors [m, feature_dim]
        lengthscale: Length scale parameter
        variance: Variance parameter
        nu: Smoothness parameter (1.5 or 2.5)
    """
    # Scale features by lengthscale
    x1_scaled = x1 / lengthscale
    x2_scaled = x2 / lengthscale

    # Compute squared distance
    r2 = torch.cdist(x1_scaled, x2_scaled, p=2)**2
    r = torch.sqrt(r2 + 1e-12)  # Add small epsilon for numerical stability

    # Apply Matern kernel based on nu
    if nu == 1.5:
        sqrt_3 = np.sqrt(3)
        k = variance * (1.0 + sqrt_3 * r) * torch.exp(-sqrt_3 * r)
    elif nu == 2.5:
        sqrt_5 = np.sqrt(5)
        k = variance * (1.0 + sqrt_5 * r + 5.0/3.0 * r2) * torch.exp(-sqrt_5 * r)
    else:
        raise ValueError(f"Only nu=1.5 and nu=2.5 are implemented, got {nu}")

    return k


class SpatialModel(nn.Module):
    """
    Implementation of the Spatial Causal Model with Gaussian Process and Inducing Points
    for modeling spatially-varying causal parameters.
    """
    def __init__(self, config, total_num, sigma=0.15, geo_features=None, PLS=None, PLF=None, PBD=None):
        super(SpatialModel, self).__init__()
        self.batch_size = config.batch_size
        self.geo_features_dim = config.geofeature_dim
        self.hidden_dim = config.hidden_dim
        self.num_inducing = config.num_inducing
        self.num_flows = config.num_flows
        self.z_size = config.z_size
        self.device = config.device
        self.jitter = config.jitter
        self.optimize_inducing = config.optimize_inducing

        # Ensure geo_features is on the right device
        if geo_features is not None:
            self.geo_features_for_init = geo_features.detach().to(self.device)
        else:
            self.geo_features_for_init = None

        self.sigma = sigma
        self.prune_type = config.prune_type

        # Make sure all tensors are on the same device
        if PLS is not None and PLF is not None and PBD is not None:
            flat_PLS = PLS.reshape(total_num).detach().to(self.device)
            flat_PLF = PLF.reshape(total_num).detach().to(self.device)
            flat_PBD = PBD.reshape(total_num).detach().to(self.device)

            self.global_posteriors = {
                'LS': flat_PLS,
                'LF': flat_PLF,
                'BD': flat_PBD
            }
        else:
            self.global_posteriors = {
                'LS': torch.zeros(total_num, device=self.device),
                'LF': torch.zeros(total_num, device=self.device),
                'BD': torch.zeros(total_num, device=self.device)
            }

        # Tracking for log determinant of Jacobian for each flow
        self.log_det_j_GammaLS = 0
        self.log_det_j_GammaLF = 0
        self.log_det_j_LambdaLS = 0
        self.log_det_j_LambdaLF = 0
        self.log_det_j_LambdaBD = 0
        self.log_det_j_ALS = 0  # For a_LS parameters
        self.log_det_j_ALF = 0  # For a_LF parameters
        self.log_det_j_ABD = 0  # For a_BD parameters

        # Define neural networks for GP mean functions
        self._create_mean_networks()

        # Initialize inducing points parameters
        self._init_inducing_points()

        # Initialize GP kernel parameters
        self._init_kernel_params()

        # Initialize normalizing flows
        self._init_flows()

        # Fixed parameters for the base hazard model
        self._init_base_params()

        # Move model to device
        self.to(self.device)
        # Sigmoid activation for logits
        self.m = nn.Sigmoid()


    def _create_mean_networks(self):
        """Create simpler mean functions for GP priors"""
        # Instead of a neural network, use a zero mean or a very simple transformation
        def create_simple_mean():
            return nn.Sequential(
                nn.Linear(self.geo_features_dim, 1),
                nn.Linear(1, self.z_size)
            ).to(self.device)  # Move to device immediately after creation

        # Initialize the linear layer weights to be very small
        # Initialize with POSITIVE bias - critical change
        def init_weights(m):
            if isinstance(m, nn.Linear):
                nn.init.zeros_(m.weight)
                nn.init.constant_(m.bias, 0.5)

        # Create mean networks with zero initialization
        self.mean_GammaLS = create_simple_mean()
        self.mean_GammaLF = create_simple_mean()
        self.mean_LambdaLS = create_simple_mean()
        self.mean_LambdaLF = create_simple_mean()
        self.mean_LambdaBD = create_simple_mean()
        self.mean_ALS = create_simple_mean()
        self.mean_ALF = create_simple_mean()
        self.mean_ABD = create_simple_mean()

        # Apply zero initialization
        self.mean_GammaLS.apply(init_weights)
        self.mean_GammaLF.apply(init_weights)
        self.mean_LambdaLS.apply(init_weights)
        self.mean_LambdaLF.apply(init_weights)
        self.mean_LambdaBD.apply(init_weights)
        self.mean_ALS.apply(init_weights)
        self.mean_ALF.apply(init_weights)
        self.mean_ABD.apply(init_weights)


    def _init_inducing_points(self):
        """Initialize inducing points using k-means clustering with improved variational parameter initialization"""
        if self.geo_features_for_init is not None:
            # Use k-means to find representative points
            from sklearn.cluster import KMeans

            # Flatten geo features
            geo_features_flat = self.geo_features_for_init.reshape(-1, self.geo_features_dim)

            # Get total number of points
            total_points = geo_features_flat.shape[0]

            # Sample a manageable subset (e.g., 10,000 points)
            max_sample = min(10000, total_points)
            indices = torch.randperm(total_points)[:max_sample]
            sampled_features = geo_features_flat[indices].cpu().numpy()

            print(f"Running k-means clustering on {max_sample} points to select {self.num_inducing} inducing points...")
            kmeans = KMeans(n_clusters=self.num_inducing, random_state=42, n_init=10)
            kmeans.fit(sampled_features)

            # Get cluster centers and convert to tensor
            inducing_points = torch.tensor(kmeans.cluster_centers_, dtype=torch.float32, device=self.device)
            self.inducing_points = nn.Parameter(
                inducing_points,
                requires_grad=self.optimize_inducing)  # Use the flag to decide if inducing points are optimized
            print(f"Selected inducing points with shape: {inducing_points.shape}")
        else:
            # Fallback to random initialization if no geo_features provided
            print("No geo_features provided. Using random inducing points.")
            inducing_points = torch.randn(self.num_inducing, self.geo_features_dim, device=self.device) * 0.1
            self.inducing_points = nn.Parameter(inducing_points, requires_grad=self.optimize_inducing)

        # Explicitly ensure all neural networks are on the correct device before using them
        for name, module in self.named_modules():
            if name.startswith('mean_'):
                module.to(self.device)

        # Get prior means at inducing points - useful for initialization
        # This helps align posterior means with prior means initially
        if hasattr(self, 'mean_GammaLS'):
            # Make sure inducing_points is on the right device
            inducing_points_device = self.inducing_points.to(self.device)

            prior_means = {
                'GammaLS': self.mean_GammaLS(inducing_points_device).detach(),
                'GammaLF': self.mean_GammaLF(inducing_points_device).detach(),
                'LambdaLS': self.mean_LambdaLS(inducing_points_device).detach(),
                'LambdaLF': self.mean_LambdaLF(inducing_points_device).detach(),
                'LambdaBD': self.mean_LambdaBD(inducing_points_device).detach(),
                'ALS': self.mean_ALS(inducing_points_device).detach(),
                'ALF': self.mean_ALF(inducing_points_device).detach(),
                'ABD': self.mean_ABD(inducing_points_device).detach()
            }
        else:
            # If mean networks don't exist yet, use zeros
            prior_means = {k: torch.zeros(self.num_inducing, self.z_size, device=self.device)
                          for k in ['GammaLS', 'GammaLF', 'LambdaLS', 'LambdaLF', 'LambdaBD', 'ALS', 'ALF', 'ABD']}

        # Initialize variational parameters for each GP
        # Mean vectors - initialize close to prior means
        noise_scale = 0.01  # Small noise to break symmetry
        self.mu_GammaLS = nn.Parameter(prior_means['GammaLS'] + noise_scale * torch.randn_like(prior_means['GammaLS']))
        self.mu_GammaLF = nn.Parameter(prior_means['GammaLF'] + noise_scale * torch.randn_like(prior_means['GammaLF']))
        self.mu_LambdaLS = nn.Parameter(prior_means['LambdaLS'] + noise_scale * torch.randn_like(prior_means['LambdaLS']))
        self.mu_LambdaLF = nn.Parameter(prior_means['LambdaLF'] + noise_scale * torch.randn_like(prior_means['LambdaLF']))
        self.mu_LambdaBD = nn.Parameter(prior_means['LambdaBD'] + noise_scale * torch.randn_like(prior_means['LambdaBD']))
        self.mu_ALS = nn.Parameter(prior_means['ALS'] + noise_scale * torch.randn_like(prior_means['ALS']))
        self.mu_ALF = nn.Parameter(prior_means['ALF'] + noise_scale * torch.randn_like(prior_means['ALF']))
        self.mu_ABD = nn.Parameter(prior_means['ABD'] + noise_scale * torch.randn_like(prior_means['ABD']))

        # Low-rank factors for covariance matrices - very conservative initialization
        # Use a much smaller scale to start
        scale = 0.01  # Start with very small values

        # Initialize as scaled identity matrix
        eye = torch.eye(self.num_inducing, device=self.device) * scale
        self.L_GammaLS = nn.Parameter(eye.clone())
        self.L_GammaLF = nn.Parameter(eye.clone())
        self.L_LambdaLS = nn.Parameter(eye.clone())
        self.L_LambdaLF = nn.Parameter(eye.clone())
        self.L_LambdaBD = nn.Parameter(eye.clone())
        self.L_ALS = nn.Parameter(eye.clone())
        self.L_ALF = nn.Parameter(eye.clone())
        self.L_ABD = nn.Parameter(eye.clone())

        # Diagonal elements - initialize to slightly larger than jitter
        # This ensures positive definiteness but keeps variance small
        diag_init = torch.ones(self.num_inducing, device=self.device) * (self.jitter * 10)
        self.diag_GammaLS = nn.Parameter(diag_init.clone())
        self.diag_GammaLF = nn.Parameter(diag_init.clone())
        self.diag_LambdaLS = nn.Parameter(diag_init.clone())
        self.diag_LambdaLF = nn.Parameter(diag_init.clone())
        self.diag_LambdaBD = nn.Parameter(diag_init.clone())
        self.diag_ALS = nn.Parameter(diag_init.clone())
        self.diag_ALF = nn.Parameter(diag_init.clone())
        self.diag_ABD = nn.Parameter(diag_init.clone())


    def _init_kernel_params(self):
        """Initialize GP kernel parameters with more stable values"""
        # For lengthscales, we want to start with a moderate value
        # Use a direct parameterization and ensure lengthscales are positive with softplus
        # Target initial lengthscale of 1.0 (after softplus)
        ls_target = 1.0
        ls_init = torch.log(torch.exp(torch.ones(1, device=self.device) * ls_target) - 1)

        # Initialize all lengthscales to the same value initially
        self.ls_GammaLS = nn.Parameter(ls_init.clone())
        self.ls_GammaLF = nn.Parameter(ls_init.clone())
        self.ls_LambdaLS = nn.Parameter(ls_init.clone())
        self.ls_LambdaLF = nn.Parameter(ls_init.clone())
        self.ls_LambdaBD = nn.Parameter(ls_init.clone())
        self.ls_ALS = nn.Parameter(ls_init.clone())
        self.ls_ALF = nn.Parameter(ls_init.clone())
        self.ls_ABD = nn.Parameter(ls_init.clone())

        # For variances, we want to start with a small value to prevent initial explosion
        # Target initial variance of 0.1 (after softplus)
        var_target = 0.1
        var_init = torch.log(torch.exp(torch.ones(1, device=self.device) * var_target) - 1)

        # Initialize all variances to the same conservative value
        self.var_GammaLS = nn.Parameter(var_init.clone())
        self.var_GammaLF = nn.Parameter(var_init.clone())
        self.var_LambdaLS = nn.Parameter(var_init.clone())
        self.var_LambdaLF = nn.Parameter(var_init.clone())
        self.var_LambdaBD = nn.Parameter(var_init.clone())
        self.var_ALS = nn.Parameter(var_init.clone())
        self.var_ALF = nn.Parameter(var_init.clone())
        self.var_ABD = nn.Parameter(var_init.clone())


    def _init_flows(self):
        """Initialize normalizing flows with more expressive transformations"""
        # Use larger scale for more expressive transformations
        flow_scale = 0.1

        self.flows_GammaLS = nn.ModuleList([Planar(self.z_size, scale=flow_scale) for _ in range(self.num_flows)])
        self.flows_GammaLF = nn.ModuleList([Planar(self.z_size, scale=flow_scale) for _ in range(self.num_flows)])
        self.flows_LambdaLS = nn.ModuleList([Planar(self.z_size, scale=flow_scale) for _ in range(self.num_flows)])
        self.flows_LambdaLF = nn.ModuleList([Planar(self.z_size, scale=flow_scale) for _ in range(self.num_flows)])
        self.flows_LambdaBD = nn.ModuleList([Planar(self.z_size, scale=flow_scale) for _ in range(self.num_flows)])
        self.flows_ALS = nn.ModuleList([Planar(self.z_size, scale=flow_scale) for _ in range(self.num_flows)])
        self.flows_ALF = nn.ModuleList([Planar(self.z_size, scale=flow_scale) for _ in range(self.num_flows)])
        self.flows_ABD = nn.ModuleList([Planar(self.z_size, scale=flow_scale) for _ in range(self.num_flows)])


    def _init_base_params(self):
        # Match MATLAB initialization
        self.w_0LS = nn.Parameter(torch.rand(1) * -1)  # Negated random values
        self.w_0LF = nn.Parameter(torch.zeros(1))      # Zero initialization
        self.w_0BD = nn.Parameter(torch.rand(1) * -1)  # Negated random values
        self.w_0y = nn.Parameter(torch.rand(1))        # Random values

        # Adjust noise parameters
        noise_init = torch.rand(1)  # Random 0-1 values like MATLAB
        self.w_eLS = nn.Parameter(noise_init.clone())
        self.w_eLF = nn.Parameter(noise_init.clone())
        self.w_eBD = nn.Parameter(noise_init.clone())
        self.w_ey = nn.Parameter(noise_init.clone())


    def process_gp_with_inducing_points(self, geo_features, mean_network, mu_u, L_u, diag_u, ls, var, flows):
        """
        Memory-efficient implementation with better gradient handling.
        """
        batch_size = geo_features.shape[0]
        device = geo_features.device

        # Apply softplus with gradient-friendly approach
        ls_pos = F.softplus(ls)
        var_pos = F.softplus(var)

        # 1. Compute inducing point kernel matrix (small matrix)
        # Add jitter for numerical stability
        K_ZZ = matern_kernel(self.inducing_points, self.inducing_points, ls_pos, var_pos) + \
              self.jitter * torch.eye(self.num_inducing, device=device)

        # Handle inverse operation with better numerical stability
        try:
            L_ZZ = torch.linalg.cholesky(K_ZZ)
            K_ZZ_inv = torch.cholesky_inverse(L_ZZ)
        except:
            # Fall back to SVD for more stable pseudo-inverse
            K_ZZ_inv = torch.pinverse(K_ZZ)

        # 2. Process in chunks for memory efficiency
        chunk_size = min(64, batch_size)
        num_chunks = (batch_size + chunk_size - 1) // chunk_size

        # Initialize outputs
        mean_l = torch.zeros((batch_size, 1), device=device)
        z_0 = torch.zeros((batch_size, 1), device=device)
        log_det_j = torch.zeros(batch_size, device=device)

        # Compute prior mean at inducing points
        prior_mean_Z = mean_network(self.inducing_points)

        # Precompute the inducing point term
        inducing_term = torch.matmul(K_ZZ_inv, (mu_u - prior_mean_Z))

        # Build the variational posterior covariance at inducing points
        Sigma_u = torch.matmul(L_u, L_u.transpose(-1, -2)) + torch.diag(F.softplus(diag_u))

        # Process in chunks
        for chunk_idx in range(num_chunks):
            # Get chunk indices
            start_idx = chunk_idx * chunk_size
            end_idx = min((chunk_idx + 1) * chunk_size, batch_size)
            chunk_geo = geo_features[start_idx:end_idx]

            # Compute GP posterior mean efficiently for this chunk
            prior_mean_chunk = mean_network(chunk_geo)
            K_lZ_chunk = matern_kernel(chunk_geo, self.inducing_points, ls_pos, var_pos)
            mean_l_chunk = prior_mean_chunk + torch.matmul(K_lZ_chunk, inducing_term)

            # Compute Woodbury correction term (memory-efficient)
            woodbury_term = torch.matmul(K_lZ_chunk, torch.matmul(K_ZZ_inv,
                              (torch.eye(self.num_inducing, device=device) - torch.matmul(Sigma_u, K_ZZ_inv))))

            # Diagonal variance (without forming full K_ll)
            diag_K_ll = var_pos * torch.ones(end_idx - start_idx, device=device)

            # Compute standard deviation at each point
            std_term = diag_K_ll - torch.sum(woodbury_term * K_lZ_chunk, dim=1)
            # Ensure positive standard deviation
            std_term = torch.clamp(std_term, min=1e-8)
            std_l = torch.sqrt(std_term)

            # Sample in a memory-efficient way
            eps = torch.randn(end_idx - start_idx, device=device)
            z_0_chunk = mean_l_chunk.squeeze() + std_l * eps

            # Store chunk results
            mean_l[start_idx:end_idx] = mean_l_chunk
            z_0[start_idx:end_idx, 0] = z_0_chunk

        # Transform through normalizing flows
        z_k = z_0.detach().clone()
        log_det_j = torch.zeros(batch_size, device=device)

        # Apply flows sequentially
        for flow in flows:
            # Apply flow to all data
            z_k_next, ldj = flow(z_k)
            z_k = z_k_next  # No in-place operation
            log_det_j = log_det_j + ldj  # No in-place operation

        # Calculate KL divergence
        inducing_centered = (mu_u - prior_mean_Z).squeeze()

        # Calculate KL divergence with numerically stable operations
        trace_term = torch.trace(torch.matmul(K_ZZ_inv, Sigma_u))
        quad_term = torch.matmul(inducing_centered.t(), torch.matmul(K_ZZ_inv, inducing_centered))

        # Compute log determinants
        try:
            log_det_K_ZZ = torch.logdet(K_ZZ)
            log_det_Sigma = torch.logdet(Sigma_u + self.jitter * torch.eye(self.num_inducing, device=device))
        except:
            # Fallback for numerical stability
            log_det_K_ZZ = torch.sum(torch.log(torch.linalg.eigvalsh(K_ZZ) + self.jitter))
            log_det_Sigma = torch.sum(torch.log(torch.linalg.eigvalsh(Sigma_u) + self.jitter))

        # Log KL with numerical stability
        log_kl_inducing = 0.5 * (trace_term + quad_term - self.num_inducing + log_det_K_ZZ - log_det_Sigma)

        # Ensure minimum KL value for stability
        min_kl_value = torch.tensor(0.01, device=device)
        log_kl_inducing = torch.max(log_kl_inducing, min_kl_value)

        return z_k, log_det_j, log_kl_inducing



    def calculate_spatial_smoothness_loss(self, geo_f):
        """
        Calculate spatial smoothness loss using the original GP outputs
        with improved memory efficiency.
        """
        batch_size = geo_f.shape[0]
        device = geo_f.device

        # For very large batches, use chunking
        if batch_size > 1024:
            return self._chunked_smoothness_loss(geo_f)

        # Calculate pairwise distances in feature space
        distances = torch.cdist(geo_f, geo_f, p=2)

        # Normalize distances to control the influence decay
        mean_dist = torch.mean(distances) + 1e-8
        weights = torch.exp(-distances / mean_dist)  # Gaussian kernel

        # Only use upper triangular part to avoid duplicate calculations
        mask = torch.triu(torch.ones_like(weights, dtype=bool), diagonal=1)
        weights = weights * mask

        # Calculate weighted differences between spatial coefficients
        smoothness_loss = torch.tensor(0.0, device=device)

        # For each type of spatial coefficient - use the ORIGINAL z_x_K values
        for coef_name, coef in [
            ('ALS', self.z_ALS_K), ('ALF', self.z_ALF_K), ('ABD', self.z_ABD_K),
            ('GammaLS', self.z_GammaLS_K), ('GammaLF', self.z_GammaLF_K),
            ('LambdaLS', self.z_LambdaLS_K), ('LambdaLF', self.z_LambdaLF_K),
            ('LambdaBD', self.z_LambdaBD_K)
        ]:
            if coef is not None:
                # Pairwise differences between coefficient values
                coef_squeezed = coef.squeeze().detach()
                coef_diff = (coef_squeezed.unsqueeze(1) - coef_squeezed.unsqueeze(0)).pow(2)

                # Apply mask to use only upper triangular
                coef_diff = coef_diff * mask

                # Weight by proximity (nearer locations should have more similar coefficients)
                weighted_diff = weights * coef_diff

                # Sum and normalize
                coef_loss = weighted_diff.sum()
                weight_sum = weights.sum() + 1e-8
                smoothness_loss += coef_loss / weight_sum

                # Free memory
                del coef_diff, coef_squeezed

        return smoothness_loss

    def _chunked_smoothness_loss(self, geo_f):
        """
        Helper method for chunked computation of smoothness loss when batch size is very large.
        Keeps all computations on GPU while managing memory usage.
        """
        batch_size = geo_f.shape[0]
        device = geo_f.device
        chunk_size = 256  # Can be adjusted based on GPU memory

        smoothness_loss = torch.tensor(0.0, device=device)
        num_chunks = (batch_size + chunk_size - 1) // chunk_size

        # Process each coefficient type separately
        for coef_name, coef in [
            ('ALS', self.z_ALS_K), ('ALF', self.z_ALF_K), ('ABD', self.z_ABD_K),
            ('GammaLS', self.z_GammaLS_K), ('GammaLF', self.z_GammaLF_K),
            ('LambdaLS', self.z_LambdaLS_K), ('LambdaLF', self.z_LambdaLF_K),
            ('LambdaBD', self.z_LambdaBD_K)
        ]:
            if coef is None:
                continue

            coef_loss = torch.tensor(0.0, device=device)
            weight_sum = torch.tensor(0.0, device=device)

            # Keep coefficient on GPU, just detach for gradient safety
            coef_gpu = coef.squeeze().detach()

            # Process chunks in upper triangular fashion
            for i in range(num_chunks):
                # Calculate chunk boundaries
                i_start = i * chunk_size
                i_end = min((i+1) * chunk_size, batch_size)

                # Get geo features for this chunk
                geo_chunk_i = geo_f[i_start:i_end].detach()
                coef_chunk_i = coef_gpu[i_start:i_end]

                # Process upper triangular chunks (including diagonal)
                for j in range(i, num_chunks):
                    # Calculate chunk boundaries
                    j_start = j * chunk_size
                    j_end = min((j+1) * chunk_size, batch_size)

                    # Get geo features for this chunk
                    geo_chunk_j = geo_f[j_start:j_end].detach()
                    coef_chunk_j = coef_gpu[j_start:j_end]

                    # Calculate pairwise distances between chunks
                    distances = torch.cdist(geo_chunk_i, geo_chunk_j, p=2)

                    # Normalize distances
                    mean_dist = torch.mean(distances) + 1e-8
                    weights = torch.exp(-distances / mean_dist)

                    # Handle diagonal chunks specially
                    if i == j:
                        # Create upper triangular mask for diagonal chunk
                        rows = weights.shape[0]
                        cols = weights.shape[1]
                        mask = torch.zeros((rows, cols), dtype=torch.bool, device=device)

                        # Fill upper triangular part (excluding diagonal)
                        for row in range(rows):
                            mask[row, row+1:] = True

                        # Apply mask to weights
                        weights = weights * mask

                        # Calculate coefficient differences and apply mask
                        coef_diff = (coef_chunk_i.unsqueeze(1) - coef_chunk_i.unsqueeze(0)).pow(2)
                        coef_diff = coef_diff * mask
                    else:
                        # For non-diagonal chunks, use all pairs
                        coef_diff = (coef_chunk_i.unsqueeze(1) - coef_chunk_j.unsqueeze(0)).pow(2)

                    # Calculate weighted differences
                    weighted_diff = (weights * coef_diff).sum()
                    current_weight_sum = weights.sum()

                    # Accumulate results
                    coef_loss = coef_loss + weighted_diff
                    weight_sum = weight_sum + current_weight_sum

                    # Free memory for this chunk pair
                    del distances, weights, coef_diff
                    del geo_chunk_j, coef_chunk_j

                # Free memory for this chunk
                del geo_chunk_i, coef_chunk_i

                # Periodically clear CUDA cache
                if i % 4 == 0:
                    torch.cuda.empty_cache()

            # Add normalized loss for this coefficient
            if weight_sum > 0:
                smoothness_loss = smoothness_loss + (coef_loss / weight_sum)

            # Free memory
            del coef_gpu, coef_loss, weight_sum
            torch.cuda.empty_cache()

        return smoothness_loss





    def calculate_em_posterior_updates(self, a_ls, a_lf, a_bd, obs, z_LS, z_LF, z_BD, scale=5, temperature=1.0):
        """
        Calculate EM algorithm posterior updates with straight-through estimator.
        This maintains gradient flow while properly implementing the Tfxn logic from MATLAB.

        Args:
            a_ls, a_lf, a_bd: Prior knowledge for hazards
            obs: Observations (damage proxy maps)
            z_LS, z_LF, z_BD: Current posteriors
            scale: Scaling factor for causal coefficients
            temperature: Temperature for sigmoid to prevent saturation

        Returns:
            Updated posteriors with gradient path preserved
        """
        # Store original tensors for gradient path
        z_LS_original = z_LS
        z_LF_original = z_LF
        z_BD_original = z_BD

        # Execute EM update without gradients
        with torch.no_grad():
            # Create detached copies to break the gradient graph
            z_LS_local = z_LS.detach().clone()
            z_LF_local = z_LF.detach().clone()
            z_BD_local = z_BD.detach().clone()

            # Helper function similar to f in MATLAB code
            def f(a):
                return -torch.log(1 + torch.exp(a))

            # Get the current parameters (detached for stability)
            noise_y = F.softplus(self.w_ey.detach())
            noise_LS = F.softplus(self.w_eLS.detach())
            noise_LF = F.softplus(self.w_eLF.detach())
            noise_BD = F.softplus(self.w_eBD.detach())

            # w_scale = 5
            w = [
                self.w_0y.detach(),
                F.softplus(self.w_ey.detach()),
                self.w_0BD.detach(),
                self.w_0LS.detach(),
                self.w_0LF.detach(),
                self.z_GammaLS_K.squeeze().detach(),     # Removed scale
                self.z_GammaLF_K.squeeze().detach(),     # Removed scale
                self.z_LambdaBD_K.squeeze().detach(),    # Removed scale
                self.z_LambdaLS_K.squeeze().detach(),    # Removed scale
                self.z_LambdaLF_K.squeeze().detach(),    # Removed scale
                F.softplus(self.w_eLS.detach()),
                F.softplus(self.w_eLF.detach()),
                F.softplus(self.w_eBD.detach()),
                self.z_ALS_K.squeeze().detach(),         # Removed scale
                self.z_ALF_K.squeeze().detach(),         # Removed scale
                self.z_ABD_K.squeeze().detach()
            ]

            # Get local mask
            batch_size = z_LS_local.size(0)
            local_mask = getattr(self, 'local_mask', torch.ones(batch_size, device=self.device)).detach()

            # Log transform observations
            log_y = torch.log(torch.clamp(obs.detach(), min=1e-6))

            # Initialize arrays for EM updates
            g_BD = torch.zeros_like(z_BD_local)
            g_LS = torch.zeros_like(z_LS_local)
            g_LF = torch.zeros_like(z_LF_local)

            delta = 0.01  # Small value for stability

            # Create masks for different local model classes
            local_3_mask = (local_mask == 3).float()
            local_4_mask = (local_mask == 4).float()
            local_3_4_6_mask = ((local_mask == 3) | (local_mask == 4) | (local_mask == 6)).float()
            local_6_mask = (local_mask == 6).float()
            local_1_3_5_6_mask = ((local_mask == 1) | (local_mask == 3) | (local_mask == 5) | (local_mask == 6)).float()
            local_5_mask = (local_mask == 5).float()
            local_5_6_mask = ((local_mask == 5) | (local_mask == 6)).float()
            local_2_4_5_6_mask = ((local_mask == 2) | (local_mask == 4) | (local_mask == 5) | (local_mask == 6)).float()
            local_4_6_mask = ((local_mask == 4) | (local_mask == 6)).float()

            # Calculate g_BD components (following Tfxn MATLAB implementation)
            # For BD (building damage)
            g_BD += local_3_4_6_mask * (
                f(-w[2] - w[15] * a_bd + (w[13]**2)/2) -
                f(w[2] + w[15] * a_bd + (w[13]**2)/2)
            )

            # Local 3 component (Landslide to BD)
            g_BD += local_3_mask * (
                z_LS_local * f(-w[2] - w[5] + (w[12]**2)/2) +
                (1-z_LS_local) * f(-w[2] + (w[12]**2)/2) -
                z_LS_local * f(w[2] + w[5] + (w[12]**2)/2) -
                (1-z_LS_local) * f(w[2] + (w[12]**2)/2)
            )

            # Local 4 component (Liquefaction to BD)
            g_BD += local_4_mask * (
                z_LF_local * f(-w[2] - w[6] + (w[12]**2)/2) +
                (1-z_LF_local) * f(-w[2] + (w[12]**2)/2) -
                z_LF_local * f(w[2] + w[6] + (w[12]**2)/2) -
                (1-z_LF_local) * f(w[2] + (w[12]**2)/2)
            )

            # Local 3,4,6 component (DPM effect)
            g_BD -= local_3_4_6_mask * ((w[7] - 2*log_y + 2*w[0]) / (2*(w[1]**2))) * w[7]

            # Local 3 component (LS interaction)
            g_BD -= local_3_mask * (1/(w[1]**2)) * (w[8] * w[7] * z_LS_local)

            # Local 4 component (LF interaction)
            g_BD -= local_4_mask * (1/(w[1]**2)) * (w[9] * w[7] * z_LF_local)

            # Local 6 component (LS and LF interaction)
            g_BD -= local_6_mask * (1/(w[1]**2)) * (w[8] * w[9] * w[7] * z_LS_local * z_LF_local)

            # Local 6 component (complex interactions from MATLAB)
            g_BD -= local_6_mask * (
                (z_LS_local * z_LF_local * f(-w[2] - w[5] - w[6] + (w[12]**2)/2) +
                z_LS_local * (1-z_LF_local) * f(-w[2] - w[5] + (w[12]**2)/2) +
                (1-z_LS_local) * z_LF_local * f(-w[2] - w[6] + (w[12]**2)/2) -
                (1-z_LS_local) * (1-z_LF_local) * f(w[2] + (w[12]**2)/2)) -
                (z_LS_local * z_LF_local * f(w[2] + w[5] + w[6] + (w[12]**2)/2) -
                z_LS_local * (1-z_LF_local) * f(w[2] + w[5] + (w[12]**2)/2) -
                (1-z_LS_local) * z_LF_local * f(w[2] + w[6] + (w[12]**2)/2) -
                (1-z_LS_local) * (1-z_LF_local) * f(w[2] + (w[12]**2)/2))
            )

            # For LS (landslide)
            # Components of gLS - using the correct w[13] coefficient for a_ls (MATLAB w(14))
            g_LS += local_1_3_5_6_mask * (
                f(-w[3] - w[13] * a_ls + (w[10]**2)/2) -
                f(w[3] + w[13] * a_ls + (w[10]**2)/2)
            )

            g_LS += local_3_mask * z_BD_local * (
                f(-w[2] - w[5] + (w[12]**2)/2) - f(-w[2] + (w[12]**2)/2)
            )

            g_LS += local_3_mask * (1-z_BD_local) * (
                f(w[2] + w[5] + (w[12]**2)/2) - f(w[2] + (w[12]**2)/2)
            )

            g_LS += local_6_mask * z_BD_local * (
                z_LF_local * f(-w[2] - w[5] - w[6] + (w[12]**2)/2) +
                (1-z_LF_local) * f(-w[2] - w[5] + (w[12]**2)/2) -
                z_LF_local * f(-w[2] - w[6] + (w[12]**2)/2) -
                (1-z_LF_local) * f(-w[2] + (w[12]**2)/2)
            )

            g_LS += local_6_mask * (1-z_BD_local) * (
                z_LF_local * f(w[2] + w[5] + w[6] + (w[12]**2)/2) +
                (1-z_LF_local) * f(w[2] + w[5] + (w[12]**2)/2) -
                z_LF_local * f(w[2] + w[6] + (w[12]**2)/2) -
                (1-z_LF_local) * f(w[2] + (w[12]**2)/2)
            )

            g_LS -= local_1_3_5_6_mask * ((w[8] - 2*log_y + 2*w[0]) / (2*(w[1]**2))) * w[8]

            g_LS -= local_3_mask * ((w[7]*w[8]*z_BD_local) / (w[1]**2))

            g_LS -= local_5_mask * ((w[8]*w[9]*z_LF_local) / (w[1]**2))

            g_LS -= local_6_mask * ((w[7]*w[8]*w[9]*z_LF_local*z_BD_local) / (w[1]**2))

            g_LS -= local_5_6_mask * z_LF_local / (2*delta)

            # For LF (liquefaction)
            # Components of gLF - using the correct w[14] coefficient for a_lf (MATLAB w(15))
            g_LF += local_2_4_5_6_mask * (
                f(-w[4] - w[14] * a_lf + (w[11]**2)/2) -
                f(w[4] + w[14] * a_lf + (w[11]**2)/2)
            )
            # print(f"Check 1: g_LF Mask 1 min/mean/max: {local_2_4_5_6_mask.min().item():.4f}/{local_2_4_5_6_mask.mean().item():.4f}/{local_2_4_5_6_mask.max().item():.4f}")
            # print(f"Check 1: g_LF min/mean/max: {g_LF.min().item():.4f}/{g_LF.mean().item():.4f}/{g_LF.max().item():.4f}")

            g_LF += local_4_6_mask * z_BD_local * (
                f(-w[2] - w[6] + (w[12]**2)/2) - f(-w[2] + (w[12]**2)/2)
            )

            # print(f"Check 2: g_LF Mask 2 min/mean/max: {local_4_6_mask.min().item():.4f}/{local_4_6_mask.mean().item():.4f}/{local_4_6_mask.max().item():.4f}")
            # print(f"Check 2: g_LF min/mean/max: {g_LF.min().item():.4f}/{g_LF.mean().item():.4f}/{g_LF.max().item():.4f}")



            g_LF += local_4_6_mask * (1-z_BD_local) * (
                f(w[2] + w[6] + (w[12]**2)/2) - f(w[2] + (w[12]**2)/2)
            )

            # print(f"Check 3: g_LF Mask 3 min/mean/max: {local_4_6_mask.min().item():.4f}/{local_4_6_mask.mean().item():.4f}/{local_4_6_mask.max().item():.4f}")
            # print(f"Check 3: g_LF min/mean/max: {g_LF.min().item():.4f}/{g_LF.mean().item():.4f}/{g_LF.max().item():.4f}")


            g_LF += local_6_mask * z_BD_local * (
                z_LS_local * f(-w[2] - w[5] - w[6] + (w[12]**2)/2) +
                (1-z_LS_local) * f(-w[2] - w[6] + (w[12]**2)/2) -
                z_LS_local * f(-w[2] - w[5] + (w[12]**2)/2) -
                (1-z_LS_local) * f(-w[2] + (w[12]**2)/2)
            )

            # print(f"Check 4: g_LF Mask 4 min/mean/max: {local_6_mask.min().item():.4f}/{local_6_mask.mean().item():.4f}/{local_6_mask.max().item():.4f}")
            # print(f"Check 4: g_LF min/mean/max: {g_LF.min().item():.4f}/{g_LF.mean().item():.4f}/{g_LF.max().item():.4f}")


            g_LF += local_6_mask * (1-z_BD_local) * (
                z_LS_local * f(w[2] + w[5] + w[6] + (w[12]**2)/2) +
                (1-z_LS_local) * f(w[2] + w[6] + (w[12]**2)/2) -
                z_LS_local * f(w[2] + w[5] + (w[12]**2)/2) -
                (1-z_LS_local) * f(w[2] + (w[12]**2)/2)
            )
            # print(f"Check 5: g_LF Mask 5 min/mean/max: {local_6_mask.min().item():.4f}/{local_6_mask.mean().item():.4f}/{local_6_mask.max().item():.4f}")
            # print(f"Check 5: g_LF min/mean/max: {g_LF.min().item():.4f}/{g_LF.mean().item():.4f}/{g_LF.max().item():.4f}")



            g_LF -= local_2_4_5_6_mask * ((w[9] - 2*log_y + 2*w[0]) / (2*(w[1]**2))) * w[9]

            g_LF -= local_4_mask * ((w[7]*w[9]*z_BD_local) / (w[1]**2))

            g_LF -= local_5_mask * ((w[8]*w[9]*z_LS_local) / (w[1]**2))

            g_LF -= local_6_mask * ((w[7]*w[8]*w[9]*z_LS_local*z_BD_local) / (w[1]**2))

            g_LF -= local_5_6_mask * z_LS_local / (2*delta)


            # After calculating g_LS, g_LF, g_BD
            # Add normalization to limit gradient magnitudes
            g_LS = torch.clamp(g_LS, min=-50.0, max=10.0)
            g_LF = torch.clamp(g_LF, min=-50.0, max=10.0)
            g_BD = torch.clamp(g_BD, min=-50.0, max=10.0)

            # Print diagnostic information about g values
            if torch.isnan(g_LF).any() or torch.isnan(g_LS).any() or torch.isnan(g_BD).any():
                print("WARNING: NaN values detected in g_LF, g_LS, or g_BD")

            # print(f"Batch size: {z_LS.size(0)}")

            # After creating local masks
            # print(f"LS Mask sum: {local_1_3_5_6_mask.sum().item()}")
            # print(f"LF Mask sum: {local_2_4_5_6_mask.sum().item()}")
            # print(f"BD Mask sum: {local_3_4_6_mask.sum().item()}")

            # print(f"g_LS min/mean/max: {g_LS.min().item():.4f}/{g_LS.mean().item():.4f}/{g_LS.max().item():.4f}")
            # print(f"g_LF min/mean/max: {g_LF.min().item():.4f}/{g_LF.mean().item():.4f}/{g_LF.max().item():.4f}")
            # print(f"g_BD min/mean/max: {g_BD.min().item():.4f}/{g_BD.mean().item():.4f}/{g_BD.max().item():.4f}")

            # Apply sigmoid with temperature to get updated posteriors
            new_z_BD = torch.sigmoid(g_BD / temperature)
            new_z_LS = torch.sigmoid(g_LS / temperature)
            new_z_LF = torch.sigmoid(g_LF / temperature)

            # Apply masks to ensure zeros where needed - this preserves exact zeros
            new_z_LS = new_z_LS * local_1_3_5_6_mask
            new_z_LF = new_z_LF * local_2_4_5_6_mask
            new_z_BD = new_z_BD * local_3_4_6_mask

            # Only for non-zero locations, ensure values aren't exactly 1
            # This prevents log(0) issues while respecting deterministic zeros
            eps = 1e-6  # Small epsilon only for preventing exact 1's
            non_zero_LS = (new_z_LS > 0)
            non_zero_LF = (new_z_LF > 0)
            non_zero_BD = (new_z_BD > 0)

            new_z_LS[non_zero_LS] = torch.clamp(new_z_LS[non_zero_LS], max=1-eps)
            new_z_LF[non_zero_LF] = torch.clamp(new_z_LF[non_zero_LF], max=1-eps)
            new_z_BD[non_zero_BD] = torch.clamp(new_z_BD[non_zero_BD], max=1-eps)

        # Apply straight-through estimator: use new values but maintain gradient path
        final_z_LS = z_LS_original - z_LS_original.detach() + new_z_LS
        final_z_LF = z_LF_original - z_LF_original.detach() + new_z_LF
        final_z_BD = z_BD_original - z_BD_original.detach() + new_z_BD

        # print(f"final_z_LS min/mean/max: {final_z_LS.min().item():.4f}/{final_z_LS.mean().item():.4f}/{final_z_LS.max().item():.4f}")
        # print(f"final_z_LF min/mean/max: {final_z_LF.min().item():.4f}/{final_z_LF.mean().item():.4f}/{final_z_LF.max().item():.4f}")
        # print(f"final_z_BD min/mean/max: {final_z_BD.min().item():.4f}/{final_z_BD.mean().item():.4f}/{final_z_BD.max().item():.4f}")

        return final_z_LS, final_z_LF, final_z_BD

    def update_global_posteriors(self, IND_batch, z_LS, z_LF, z_BD):
        """Update global posteriors without in-place operations"""
        # Create new tensors for each posterior
        new_LS = self.global_posteriors['LS'].clone()
        new_LF = self.global_posteriors['LF'].clone()
        new_BD = self.global_posteriors['BD'].clone()

        # Make sure all tensors are on same device
        z_LS = z_LS.to(new_LS.device)
        z_LF = z_LF.to(new_LF.device)
        z_BD = z_BD.to(new_BD.device)


        # Update the new tensors
        new_LS[IND_batch] = z_LS
        new_LF[IND_batch] = z_LF
        new_BD[IND_batch] = z_BD

        # Replace the global posteriors with the new tensors
        self.global_posteriors = {
            'LS': new_LS,
            'LF': new_LF,
            'BD': new_BD
        }








    def forward(self, a_LS, a_LF, a_BD, x, geo_Features, ft, local, IND_batch, em_steps=1):
        device = self.device

        a_LS = a_LS.to(device)
        a_LF = a_LF.to(device)
        a_BD = a_BD.to(device)
        x = x.to(device)
        geo_Features = geo_Features.to(device)
        local = local.to(device)

        ft = ft.to(device)



        if IND_batch is not None:
            if isinstance(IND_batch, np.ndarray):
                IND_batch = torch.tensor(IND_batch, dtype=torch.long, device=device)
            else:
                IND_batch = IND_batch.to(device)
            batch_size = x.size(1)

        # Reshape inputs
        geo_f = geo_Features.view(batch_size, -1)
        obs = x.view(batch_size, -1).squeeze()
        a_ls = a_LS.view(batch_size, -1).squeeze()
        a_lf = a_LF.view(batch_size, -1).squeeze()
        a_bd = a_BD.view(batch_size, -1).squeeze()

        if hasattr(self, 'original_local'):
            mask_local = self.original_local[IND_batch].to(self.device)
        else:
            mask_local = local.clone()

        # Create masks using mask_local
        local_mask = mask_local.view(batch_size, -1).squeeze().long()
        bd_local_mask = ((local_mask == 3) | (local_mask == 4) | (local_mask == 6)).float() # 3/4/6
        bd_ft_mask = (ft != 0).float()  # Building footprint mask
        local_3_mask = (local_mask == 3).float()
        local_4_mask = (local_mask == 4).float()
        local_6_mask = (local_mask == 6).float()
        local_0_1_2_5_mask = ((local_mask == 0) | (local_mask == 1) | (local_mask == 2) | (local_mask == 5)).float() # 0/1/2/5
        # Mask for LS
        ls_local_mask = ((local_mask == 1) | (local_mask == 3) | (local_mask == 5) | (local_mask == 6)).float() # 1/3/5/6
        local_0_2_4_mask = ((local_mask == 0) | (local_mask == 2) | (local_mask == 4)).float() # 0/2/4
        # Mask for LF
        lf_local_mask = ((local_mask == 2) | (local_mask == 4) | (local_mask == 5) | (local_mask == 6)).float() # 2/4/5/6
        local_0_1_3_mask = ((local_mask == 0) | (local_mask == 1) | (local_mask == 3)).float() # 0/1/3
        # Mask for eps
        local_5_mask = (local == 5).float()
        local_6_mask = (local == 6).float()
        local_5_6_mask = ((local_mask == 5) | (local_mask == 6)).float() # 5/6

        # print(f"forward LS Mask sum: {ls_local_mask.sum().item()}")
        # print(f"forward LF Mask sum: {lf_local_mask.sum().item()}")
        # print(f"forward BD Mask sum: {bd_local_mask.sum().item()}")


        # Create masks from feature type and local model
        local_mask = local.view(batch_size, -1).squeeze()

        # CRITICAL FIX: Convert to integer type for exact comparisons
        local_mask = local_mask.long()

        self.local_mask = local_mask  # Store for use in EM updates



        # Store the masks as attributes
        self.ls_local_mask = ls_local_mask
        self.lf_local_mask = lf_local_mask
        self.bd_local_mask = bd_local_mask
        self.bd_ft_mask = bd_ft_mask



        # Process influence parameters (ALS, ALF, ABD)
        z_ALS_K, log_det_j_ALS, kl_ALS = self.process_gp_with_inducing_points(
            geo_f, self.mean_ALS, self.mu_ALS, self.L_ALS, self.diag_ALS,
            self.ls_ALS, self.var_ALS, self.flows_ALS
        )

        z_ALF_K, log_det_j_ALF, kl_ALF = self.process_gp_with_inducing_points(
            geo_f, self.mean_ALF, self.mu_ALF, self.L_ALF, self.diag_ALF,
            self.ls_ALF, self.var_ALF, self.flows_ALF
        )

        z_ABD_K, log_det_j_ABD, kl_ABD = self.process_gp_with_inducing_points(
            geo_f, self.mean_ABD, self.mu_ABD, self.L_ABD, self.diag_ABD,
            self.ls_ABD, self.var_ABD, self.flows_ABD
        )

        # Process causal strength parameters (Gamma and Lambda)
        z_GammaLS_K, log_det_j_GammaLS, kl_GammaLS = self.process_gp_with_inducing_points(
            geo_f, self.mean_GammaLS, self.mu_GammaLS, self.L_GammaLS, self.diag_GammaLS,
            self.ls_GammaLS, self.var_GammaLS, self.flows_GammaLS
        )

        z_GammaLF_K, log_det_j_GammaLF, kl_GammaLF = self.process_gp_with_inducing_points(
            geo_f, self.mean_GammaLF, self.mu_GammaLF, self.L_GammaLF, self.diag_GammaLF,
            self.ls_GammaLF, self.var_GammaLF, self.flows_GammaLF
        )

        z_LambdaLS_K, log_det_j_LambdaLS, kl_LambdaLS = self.process_gp_with_inducing_points(
            geo_f, self.mean_LambdaLS, self.mu_LambdaLS, self.L_LambdaLS, self.diag_LambdaLS,
            self.ls_LambdaLS, self.var_LambdaLS, self.flows_LambdaLS
        )

        z_LambdaLF_K, log_det_j_LambdaLF, kl_LambdaLF = self.process_gp_with_inducing_points(
            geo_f, self.mean_LambdaLF, self.mu_LambdaLF, self.L_LambdaLF, self.diag_LambdaLF,
            self.ls_LambdaLF, self.var_LambdaLF, self.flows_LambdaLF
        )

        z_LambdaBD_K, log_det_j_LambdaBD, kl_LambdaBD = self.process_gp_with_inducing_points(
            geo_f, self.mean_LambdaBD, self.mu_LambdaBD, self.L_LambdaBD, self.diag_LambdaBD,
            self.ls_LambdaBD, self.var_LambdaBD, self.flows_LambdaBD
        )

        # Apply masks to causal parameters to ensure consistency
        # Parameters for/affecting LS
        z_ALS_K = z_ALS_K * ls_local_mask.unsqueeze(-1)
        z_LambdaLS_K = z_LambdaLS_K * ls_local_mask.unsqueeze(-1)

        # Parameters for/affecting LF
        z_ALF_K = z_ALF_K * lf_local_mask.unsqueeze(-1)
        z_LambdaLF_K = z_LambdaLF_K * lf_local_mask.unsqueeze(-1)

        # Parameters for/affecting BD
        z_ABD_K = z_ABD_K * bd_local_mask.unsqueeze(-1)
        z_LambdaBD_K = z_LambdaBD_K * bd_local_mask.unsqueeze(-1)

        # LS→BD effect only exists where both LS and BD can exist
        ls_bd_mask = (ls_local_mask * bd_local_mask).unsqueeze(-1)
        z_GammaLS_K = z_GammaLS_K * ls_bd_mask

        # LF→BD effect only exists where both LF and BD can exist
        lf_bd_mask = (lf_local_mask * bd_local_mask).unsqueeze(-1)
        z_GammaLF_K = z_GammaLF_K * lf_bd_mask

        # Store the masked GP outputs for later use
        self.z_ALS_K = z_ALS_K
        self.z_ALF_K = z_ALF_K
        self.z_ABD_K = z_ABD_K
        self.z_GammaLS_K = z_GammaLS_K
        self.z_GammaLF_K = z_GammaLF_K
        self.z_LambdaLS_K = z_LambdaLS_K
        self.z_LambdaLF_K = z_LambdaLF_K
        self.z_LambdaBD_K = z_LambdaBD_K

        # Define the w parameter vector
        # w_scale = 5
        # w = [
        #     self.w_0y.detach(),
        #     F.softplus(self.w_ey.detach()),
        #     self.w_0BD.detach(),
        #     self.w_0LS.detach(),
        #     self.w_0LF.detach(),
        #     w_scale * torch.tanh(self.z_GammaLS_K.squeeze().detach()),     # Removed scale
        #     w_scale * torch.tanh(self.z_GammaLF_K.squeeze().detach()),     # Removed scale
        #     self.z_LambdaBD_K.squeeze().detach(),    # Removed scale
        #     self.z_LambdaLS_K.squeeze().detach(),    # Removed scale
        #     self.z_LambdaLF_K.squeeze().detach(),    # Removed scale
        #     F.softplus(self.w_eLS.detach()),
        #     F.softplus(self.w_eLF.detach()),
        #     F.softplus(self.w_eBD.detach()),
        #     w_scale * torch.tanh(self.z_ALS_K.squeeze().detach()),         # Removed scale
        #     w_scale * torch.tanh(self.z_ALF_K.squeeze().detach()),         # Removed scale
        #     w_scale * torch.tanh(self.z_ABD_K.squeeze().detach())
        # ]

        w = [
            self.w_0y.detach(),
            F.softplus(self.w_ey.detach()),
            self.w_0BD.detach(),
            self.w_0LS.detach(),
            self.w_0LF.detach(),
            self.z_GammaLS_K.squeeze().detach(),     # Removed scale
            self.z_GammaLF_K.squeeze().detach(),     # Removed scale
            self.z_LambdaBD_K.squeeze().detach(),    # Removed scale
            self.z_LambdaLS_K.squeeze().detach(),    # Removed scale
            self.z_LambdaLF_K.squeeze().detach(),    # Removed scale
            F.softplus(self.w_eLS.detach()),
            F.softplus(self.w_eLF.detach()),
            F.softplus(self.w_eBD.detach()),
            self.z_ALS_K.squeeze().detach(),         # Removed scale
            self.z_ALF_K.squeeze().detach(),         # Removed scale
            self.z_ABD_K.squeeze().detach()
        ]
        # Convert IND_batch into a PyTorch tensor
        if isinstance(IND_batch, np.ndarray):
            IND_batch = torch.tensor(IND_batch, dtype=torch.long, device=self.device)
        else:
            IND_batch = IND_batch.to(device=self.device)

        # Retrieve stored posteriors (or use prior values if missing)
        stored_ls = self.global_posteriors['LS'][IND_batch]
        stored_lf = self.global_posteriors['LF'][IND_batch]
        stored_bd = self.global_posteriors['BD'][IND_batch]

        # Use prior model if stored posteriors are NaN
        z_LS = torch.where(torch.isnan(stored_ls), a_LS.view(batch_size), stored_ls)
        z_LF = torch.where(torch.isnan(stored_lf), a_LF.view(batch_size), stored_lf)
        z_BD = torch.where(torch.isnan(stored_bd), a_BD.view(batch_size), stored_bd)

        # Apply masks to posteriors
        z_LS = z_LS * ls_local_mask
        z_LF = z_LF * lf_local_mask
        z_BD = z_BD * bd_local_mask

        # Perform EM steps to update posteriors iteratively
        for step in range(em_steps):
            z_LS, z_LF, z_BD = self.calculate_em_posterior_updates(a_ls, a_lf, a_bd, obs, z_LS, z_LF, z_BD)

            # Re-apply masks after each EM update to maintain consistency
            z_LS = z_LS * ls_local_mask
            z_LF = z_LF * lf_local_mask
            z_BD = z_BD * bd_local_mask

        # Update global storage with new posterior values
        self.update_global_posteriors(IND_batch, z_LS, z_LF, z_BD)

        # After EM iterations, update local model classification
        tqLS = z_LS.detach().cpu().numpy()
        tqLF = z_LF.detach().cpu().numpy()
        local_np = local_mask.detach().cpu().numpy()

        # Find locations with ambiguous classification (class 5 or 6)
        sigma = self.sigma  # Get pruning threshold
        prune_type = self.prune_type  # Get pruning type

        ambiguous_mask = (local_np == 5) | (local_np == 6)
        if ambiguous_mask.sum() > 0:
            if prune_type == "single":
                # Single threshold pruning
                local_np[(tqLF < tqLS) & (local_np == 5)] = 1
                local_np[(tqLF < tqLS) & (local_np == 6)] = 3
                local_np[(tqLS < tqLF - sigma) & (local_np == 5)] = 2
                local_np[(tqLS < tqLF - sigma) & (local_np == 6)] = 4
            else:  # "double" pruning
                local_np[(tqLF < tqLS - sigma) & (local_np == 5)] = 1
                local_np[(tqLF < tqLS - sigma) & (local_np == 6)] = 3
                local_np[(tqLS < tqLF - sigma) & (local_np == 5)] = 2
                local_np[(tqLS < tqLF - sigma) & (local_np == 6)] = 4




        # Calculate spatial smoothness loss
        spatial_loss = self.calculate_spatial_smoothness_loss(geo_f)

        # Helper functions
        def safe_exp(x, max_val=20.0):
            return torch.exp(torch.clamp(x, max=max_val))

        def safe_mul(*args, max_val=1e10):
            result = args[0]
            for arg in args[1:]:
                result = torch.clamp(result * arg, max=max_val)
            return result

        # Compute log p(y|x,z) term in ELBO
        # Compute noise variances
        noise_var_LS = torch.clamp(F.softplus(self.w_eLS)**2 / 2, min=1e-8, max=10.0)
        noise_var_LF = torch.clamp(F.softplus(self.w_eLF)**2 / 2, min=1e-8, max=10.0)
        noise_var_BD = torch.clamp(F.softplus(self.w_eBD)**2 / 2, min=1e-8, max=10.0)
        variance = torch.clamp(F.softplus(self.w_ey)**2, min=1e-8, max=10.0)


  ############################
        # For DPM observations
        log_y = torch.log(torch.clamp(obs, min=1e-8))

        # Clamp the q values to avoid log(0)
        f_qBD = torch.clamp(z_BD + 1e-6, min=1e-6, max=1-1e-6)
        f_qLS = torch.clamp(z_LS + 1e-6, min=1e-6, max=1-1e-6)
        f_qLF = torch.clamp(z_LF + 1e-6, min=1e-6, max=1-1e-6)


        # Create masks for different local models


        def safe_log(x, eps=1e-8):
            return torch.log(torch.clamp(x, min=eps))

        def f(x):
            return -torch.log(1 + torch.exp(x))

        # Mask for BD



        # LBD calculation
        LBD_part1 = local_3_mask * z_BD * (z_LS * (f(- w[2] - w[5] + (w[12] * w[12])/2)) + (1 - z_LS) * (f(-w[2] + (w[12] * w[12])/2)))

        LBD_part2 = local_4_mask * z_BD * (z_LF * (f(- w[2] - w[6] + (w[12] * w[12])/2)) + (1 - z_LF) * (f(-w[2] + (w[12]*w[12])/2)))

        LBD_part3 = local_3_mask * (1 - z_BD) * (z_LS * (f(w[2] + w[5] + (w[12] * w[12])/2)) + (1 - z_LS) * (f(w[2] + (w[12] * w[12])/2)))

        LBD_part4 = local_4_mask * (1 - z_BD) * (z_LF * (f(w[2] + w[6] + (w[12] * w[12])/2)) + (1 - z_LF) * (f(w[2] + (w[12] * w[12])/2)))

        LBD_part5 = local_6_mask * (z_BD * (z_LS * z_LF * (f(- w[2] - w[5] - w[6] + (w[12] * w[12])/2)) +
                                z_LS * (1 - z_LF) * (f(- w[2] - w[5] + (w[12] * w[12])/2)) +
                                (1 - z_LS) * z_LF * (f(- w[2] - w[6] + (w[12] * w[12])/2)) +
                                (1 - z_LS) * (1 - z_LF) * (f(- w[2] + (w[12] * w[12])/2))) +
                                  (1 - z_BD) * (z_LS * z_LF * (f( w[2] + w[5] + w[6] + (w[12] * w[12])/2)) +
                                                z_LS * (1 - z_LF) * (f(w[2] + w[5] + (w[12]*w[12])/2)) +
                                                (1 - z_LS) * z_LF * (f(w[2] + w[6] + (w[12]*w[12])/2)) +
                                                (1 - z_LS) * (1 - z_LF) * (f(w[2] + (w[12]*w[12])/2))))

        LBD_part6 = bd_local_mask * (z_BD * safe_log(f_qBD) + (1 - z_BD) * safe_log(1-f_qBD))

        LBD = LBD_part1 + LBD_part2 + LBD_part3 + LBD_part4 + LBD_part5 - LBD_part6

        LBD = LBD * (1 - local_0_1_2_5_mask.bool().float())


        # LLS calculation
        LLS = ls_local_mask * (z_LS * f(-w[3] - w[13] * a_ls + (w[10] * w[10])/2) +
        (1 - z_LS) * f(w[3] + w[13] * a_ls + (w[10] * w[10])/2) -
                        z_LS * safe_log(f_qLS) - (1 - z_LS) * safe_log(1 - f_qLS))

        # Zero out LLS where LS is not allowed
        LLS = LLS * (1 - local_0_2_4_mask.bool().float())

        # LLF calculation
        LLF = lf_local_mask * (z_LF * f(-w[4] - w[14] * a_lf + (w[11] * w[11])/2) +
                        (1 - z_LF) * f(w[4] + w[14] * a_lf + (w[11] * w[11])/2) -
                        z_LF * safe_log(f_qLF) - (1 - z_LF) * safe_log(1 - f_qLF))

        # Zero out LLF where LF is not allowed
        LLF = LLF * (1 - local_0_1_3_mask.bool().float())



        # Leps calculation
        Leps_part1 = - (0.5 * safe_log(w[1] * w[1]) + (log_y - w[0]) * (log_y- w[0]) / (2 * (w[1] * w[1])))

        Leps_part2 = - ((1 / (2 * (w[1] * w[1]))) * (bd_local_mask * w[7] * z_BD * (w[7] - 2 * log_y + 2 * w[0]) +
                                        ls_local_mask * w[8] * z_LS * (w[8] - 2 * log_y + 2 * w[0]) +
                                        lf_local_mask * w[9] * z_LF * (w[9] - 2 * log_y + 2 * w[0])))

        Leps_part3 = - ((1 / (w[1] * w[1])) * (local_3_mask * w[7] * w[8] * z_BD * z_LS +
                                  local_4_mask * w[7] * w[9] * z_BD * z_LF +
                                  local_5_mask * w[8] * w[9] * z_LS * z_LF +
                                  local_6_mask * w[7] * w[8] * w[9] * z_BD * z_LS * z_LF))

        Leps = Leps_part1 + Leps_part2 + Leps_part3 - log_y

        # Exclusivity term
        L_ex = - local_5_6_mask * (f_qLS * f_qLF) / (2 * 1e-3)

        L_ex = torch.clamp(L_ex, min=-1e+2)  # Equivalent to max(L_ex, -1e+2)

        L = LBD + LLS + LLF + Leps + L_ex

        # KL terms for inducing points
        kl_gamma = kl_GammaLS + kl_GammaLF
        kl_lambda = kl_LambdaLS + kl_LambdaLF + kl_LambdaBD
        kl_a = kl_ALS + kl_ALF + kl_ABD

        # Total KL divergence for inducing points
        kl_inducing_points = kl_gamma + kl_lambda + kl_a
        # print("log_q_x: ", log_q_x)
        # Return all values needed for ELBO computation and model evaluation

        def ensure_batch_dim(tensor):
            if tensor.dim() == 0:
                # If tensor is a scalar, unsqueeze to add a batch dimension
                return tensor.unsqueeze(0).expand(batch_size)
            return tensor

        # After all computations, ensure outputs maintain batch dimensions
        z_LS = ensure_batch_dim(z_LS)
        z_LF = ensure_batch_dim(z_LF)
        z_BD = ensure_batch_dim(z_BD)
        L = ensure_batch_dim(L)
        return (z_LS, z_LF, z_BD,
                L, -kl_inducing_points,   # Negative KL[q(u_v)||p(u_v)]
                spatial_loss,
                local_np)              # Updated local model


from tqdm.auto import tqdm
import torch
import numpy as np

def run(MyDataLoader, model, opt, test_PLS, test_PLF, test_PBD, IND, config):
    """
    Training and evaluation function for the Spatial Causal Model with improved structure.
    """
    device = config.device

    # Check if IND matches the dataloader size
    if len(IND) != len(MyDataLoader):
        print(f"Warning: IND size ({len(IND)}) doesn't match dataloader size ({len(MyDataLoader)})")
        print(f"Using the smaller of the two to avoid index errors")

    # Training phase
    if config.epochs > 0:
        model.train()
        train_model(MyDataLoader, model, opt, IND, config)

    # Evaluation phase - generate all predictions at once
    model.eval()
    spatial_results, global_params = evaluate_model(MyDataLoader, model, test_PLS, test_PLF, test_PBD, IND, config)

    # Convert spatial_results dictionary to the original tuple format
    test_results = tuple(spatial_results[k] for k in ['PLS', 'PLF', 'PBD', 'ALS', 'ALF', 'ABD',
                                                     'GammaLS', 'GammaLF', 'LambdaLS', 'LambdaLF', 'LambdaBD'])

    return test_results, global_params

def train_model(MyDataLoader, model, opt, IND, config):
    """
    Train the Spatial Causal Model with improved error handling.
    """
    device = config.device
    model.dataloader = MyDataLoader  # Store dataloader in model for reference

    # Verify IND size matches dataloader size
    num_batches = min(len(MyDataLoader), len(IND))
    if len(IND) < len(MyDataLoader):
        print(f"WARNING: IND size ({len(IND)}) is smaller than dataloader size ({len(MyDataLoader)})")
        print(f"Will process only {num_batches} batches per epoch")

    # Training losses history
    train_losses = []

    # Track components of ELBO
    elbo_components = {
        'data_fit': [],
        'kl_inducing': [],
        'geo_penalty': [],
        'reg_loss': [],
        'total_loss': []
    }

    # Set up anomaly detection in debug mode
    if hasattr(config, 'debug') and config.debug:
        print("Anomaly detection enabled")

    print("Training progress:")
    for epoch in range(1, config.epochs + 1):

        # Track losses for this epoch
        train_loss = []
        epoch_elbo = {k: [] for k in elbo_components.keys()}

        # Create progress bar
        pbar = tqdm(total=num_batches,
                   desc=f"Epoch {epoch}/{config.epochs}",
                   bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')

        # Process mini-batches
        for batch_idx, batch_data in enumerate(MyDataLoader):
            # Stop if we've reached the end of IND
            if batch_idx >= len(IND):
                print(f"Reached end of IND array at batch {batch_idx}. Stopping iteration.")
                break

            # Compute beta for annealing
            beta = calculate_beta(epoch, batch_idx, num_batches, config)

            # Process single batch
            try:
                IND_batch = IND[batch_idx]
                batch_loss, batch_metrics = process_batch(
                    batch_data, model, opt, IND_batch, batch_idx, epoch, beta, config
                )

                # Store batch metrics if valid
                if batch_loss is not None:
                    train_loss.append(batch_loss)

                    # Store batch ELBO components
                    for k in elbo_components.keys():
                        if k in batch_metrics:
                            epoch_elbo[k].append(batch_metrics[k])

                    # Update progress bar with available metrics
                    postfix_dict = {'loss': f"{batch_loss:.4f}"}
                    for k in ['data_fit', 'kl_inducing', 'reg_loss']:
                        if k in batch_metrics:
                            postfix_dict[k] = f"{batch_metrics[k]:.2f}"
                    postfix_dict['beta'] = f"{beta:.2f}"

                    pbar.set_postfix(postfix_dict)
            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"WARNING: GPU OOM in batch {batch_idx}, skipping...")
                    # Clear cache and skip this batch
                    torch.cuda.empty_cache()
                    continue
                else:
                    print(f"Error processing batch {batch_idx}: {str(e)}")
                    if hasattr(config, 'debug') and config.debug:
                        raise  # Re-raise in debug mode
                    continue
            except IndexError as e:
                print(f"Index error in batch {batch_idx}: {str(e)}")
                print(f"IND shape: {IND.shape if hasattr(IND, 'shape') else 'unknown'}")
                print(f"batch_idx: {batch_idx}")
                continue
            except Exception as e:
                print(f"Unexpected error in batch {batch_idx}: {str(e)}")
                continue

            pbar.update(1)

            # Periodically clear GPU cache
            if batch_idx % 10 == 0:
                torch.cuda.empty_cache()

        # Close the progress bar for this epoch
        pbar.close()

        # Calculate epoch metrics if we have valid losses
        if train_loss:
            epoch_loss = sum(train_loss) / len(train_loss)
            train_losses.append(epoch_loss)

            # Calculate average ELBO components for this epoch
            for k, v in epoch_elbo.items():
                if v:  # Check if list is not empty
                    elbo_components[k].append(np.mean(v))

            # Print epoch summary
            print(f'Epoch {epoch}/{config.epochs} completed:')
            print(f'  Total Loss: {epoch_loss:.4f}')
            for k, v in elbo_components.items():
                if v:
                    print(f'  {k}: {v[-1]:.4f}')
        else:
            print(f'Epoch {epoch}/{config.epochs} completed with no valid batches')

def process_batch(batch_data, model, opt, IND_batch, batch_idx, epoch, beta, config):
    """Process a single batch for training with proper gradient handling."""

    device = config.device

    # Process batch data
    aLS, aLF, aBD, obs, geoFeatures, bdft, loc = [x.to(device) for x in batch_data]
    obs = obs + 1e-8  # Add small epsilon for numerical stability

    # Reshape inputs safely
    try:
        inputs = [aLS, aLF, aBD, obs, geoFeatures, bdft, loc]
        inputs = [x.view(-1, config.batch_size) for x in inputs]
        aLS, aLF, aBD, obs, geoF, bdft, loc = inputs
    except RuntimeError as e:
        print(f"Error reshaping batch {batch_idx}: {str(e)}")
        return None, {}

    # Clear any previous gradients
    opt.zero_grad(set_to_none=True)

    try:
        # Forward pass with fresh tensors
        with torch.set_grad_enabled(True):
            # Create deep copies of tensors to avoid shared computational history
            aLS_copy = aLS.clone()
            aLF_copy = aLF.clone()
            aBD_copy = aBD.clone()
            obs_copy = obs.clone()
            geoF_copy = geoF.clone()

            # Forward pass with copied tensors
            outputs = model(aLS_copy, aLF_copy, aBD_copy, obs_copy, geoF_copy, bdft, loc, IND_batch)

            # Unpack model outputs
            z_LS, z_LF, z_BD, L, neg_kl_inducing, spatial_loss, updated_local = outputs

            # Calculate mean of the data fit term (L)
            data_fit_mean = L.mean()

            # For negative KL, ensure it's a scalar
            if isinstance(neg_kl_inducing, torch.Tensor) and neg_kl_inducing.dim() > 0:
                neg_kl_mean = neg_kl_inducing.mean()
            else:
                neg_kl_mean = neg_kl_inducing

            # Calculate spatial loss mean
            if isinstance(spatial_loss, torch.Tensor) and spatial_loss.dim() > 0:
                spatial_loss_mean = spatial_loss.mean()
            else:
                spatial_loss_mean = spatial_loss

            # Calculate weights
            beta_data = 1
            beta_kl = 1
            spatial_weight = max(0.001, 0.01 * np.exp(-0.1 * epoch))

            # Construct loss - avoid any potential in-place operations
            loss = torch.zeros(1, device=device, requires_grad=True)

            # Add each component individually
            loss = loss - beta_data * data_fit_mean
            loss = loss - beta_kl * (neg_kl_mean / config.batch_size)
            loss = loss + spatial_weight * spatial_loss_mean

            # Add regularization - similar to MATLAB implementation
            lambda_reg = 0.01  # Regularization strength
            reg_scale = 100.0  # Scaling factor as in MATLAB
            regu_type = getattr(config, 'regu_type', 1)  # Default to 1 if not defined

            reg_loss = torch.tensor(0.0, device=device)

            if regu_type == 1:
                # Regularize Lambda parameters
                for param_name, param in [
                    ('LambdaBD', model.z_LambdaBD_K),
                    ('LambdaLS', model.z_LambdaLS_K),
                    ('LambdaLF', model.z_LambdaLF_K)
                ]:
                    if param is not None:
                        param_flat = param.squeeze().detach()
                        regu_term = lambda_reg * reg_scale * (
                            1.0 / (1.0 + torch.exp(-0.01 * param_flat)) -
                            1.0 / (1.0 + torch.exp(0.01 * param_flat))
                        )
                        reg_loss = reg_loss + regu_term.mean()
            else:
                # Regularize A parameters
                for param_name, param in [
                    ('ALS', model.z_ALS_K),
                    ('ALF', model.z_ALF_K),
                    ('ABD', model.z_ABD_K)
                ]:
                    if param is not None:
                        param_flat = param.squeeze().detach()
                        regu_term = lambda_reg * reg_scale * (
                            1.0 / (1.0 + torch.exp(-0.01 * param_flat)) -
                            1.0 / (1.0 + torch.exp(0.01 * param_flat))
                        )
                        reg_loss = reg_loss + regu_term.mean()

            # Add regularization to total loss
            loss = loss + reg_loss

            # Check for valid loss
            if torch.isnan(loss).any() or torch.isinf(loss).any():
                print(f"Invalid loss value detected in batch {batch_idx}")
                return None, {}

            # Use retain_graph to allow multiple backward passes if needed
            loss.backward(retain_graph=True)

            # Clip gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # lambda_grads = []
            # for name, param in model.named_parameters():
            #     if 'Lambda' in name and param.grad is not None:
            #         lambda_grads.append((name, param.grad.mean().item()))
            # print(f"Lambda gradients: {lambda_grads}")

            # Step optimizer
            opt.step()

            # Get scalar values
            batch_metrics = {
                'data_fit': data_fit_mean.detach().item(),
                'kl_inducing': -neg_kl_mean.detach().item() if isinstance(neg_kl_mean, torch.Tensor) else -float(neg_kl_mean),
                'spatial_loss': spatial_loss_mean.detach().item() if isinstance(spatial_loss_mean, torch.Tensor) else float(spatial_loss_mean),
                'reg_loss': reg_loss.detach().item(),
                'total_loss': loss.detach().item()
            }

            # Clean up
            del outputs, z_LS, z_LF, z_BD, L, neg_kl_inducing, spatial_loss
            del aLS_copy, aLF_copy, aBD_copy, obs_copy, geoF_copy
            torch.cuda.empty_cache()

            return loss.detach().item(), batch_metrics

    except RuntimeError as e:
        print(f"Runtime error in process_batch: {str(e)}")
        torch.cuda.empty_cache()
        return None, {}
    except Exception as e:
        print(f"Unexpected error in process_batch: {str(e)}")
        import traceback
        traceback.print_exc()
        torch.cuda.empty_cache()
        return None, {}

def calculate_beta(epoch, batch_idx, num_batches, config):
    """Calculate the annealing factor beta."""
    if epoch <= 5:
        beta = 0.01  # Start with larger value
    elif epoch <= 15:
        beta = 0.05 + (epoch - 5) * 0.03
    else:
        beta = min(0.5 + (epoch - 15) * 0.1, config.max_beta)

    return beta


def calculate_geo_penalty(model, geoF, device):
    """Calculate geospatial feature penalties."""
    avg_geo = torch.mean(geoF, dim=0)

    # Apply weights to different penalty types
    gp_weight = 0.05
    prior_weight = 0.1

    # Initialize geo_penalty
    geo_penalty = torch.tensor(0.0, device=device)

    # Get causal parameters
    causal_params = {
        'ALS': model.z_ALS_K.detach() if hasattr(model, 'z_ALS_K') else None,
        'ALF': model.z_ALF_K.detach() if hasattr(model, 'z_ALF_K') else None,
        'ABD': model.z_ABD_K.detach() if hasattr(model, 'z_ABD_K') else None,
        'GammaLS': model.z_GammaLS_K.detach() if hasattr(model, 'z_GammaLS_K') else None,
        'GammaLF': model.z_GammaLF_K.detach() if hasattr(model, 'z_GammaLF_K') else None,
        'LambdaLS': model.z_LambdaLS_K.detach() if hasattr(model, 'z_LambdaLS_K') else None,
        'LambdaLF': model.z_LambdaLF_K.detach() if hasattr(model, 'z_LambdaLF_K') else None,
        'LambdaBD': model.z_LambdaBD_K.detach() if hasattr(model, 'z_LambdaBD_K') else None
    }

    # Calculate penalties for available parameters
    for param_name, param in causal_params.items():
        if param is not None:
            weight = prior_weight if param_name in ['ALS', 'ALF', 'ABD'] else gp_weight
            penalty = torch.mean((param - avg_geo) ** 2) * weight
            geo_penalty = geo_penalty + penalty

    return geo_penalty



def evaluate_model(MyDataLoader, model, test_PLS, test_PLF, test_PBD, IND, config):
    """
    Evaluate the model on all data and generate predictions.
    """
    device = config.device
    print("Evaluating model on all data...")

    # Initialize test tensors for spatial data
    test_tensors = {
        'PLS': test_PLS.to(device),
        'PLF': test_PLF.to(device),
        'PBD': test_PBD.to(device),
        'ALS': test_PLS.clone().to(device),
        'ALF': test_PLF.clone().to(device),
        'ABD': test_PBD.clone().to(device),
        'GammaLS': test_PLS.clone().to(device),
        'GammaLF': test_PLF.clone().to(device),
        'LambdaLS': test_PLS.clone().to(device),
        'LambdaLF': test_PLF.clone().to(device),
        'LambdaBD': test_PBD.clone().to(device)
    }

    # Dictionary to store global parameters
    global_params = {}

    # Process all data in batches
    with torch.no_grad():  # No need for gradients in evaluation
        for batch_idx, batch_data in enumerate(tqdm(MyDataLoader, desc="Evaluating")):
            # Process batch data
            aLS, aLF, aBD, obs, geoFeatures, bdft, loc = [x.to(device) for x in batch_data]
            eps = 1e-8
            obs = obs + eps

            batch_size = config.batch_size
            inputs = [aLS, aLF, aBD, obs, geoFeatures, bdft, loc]
            inputs = [x.view(-1, batch_size) for x in inputs]
            aLS, aLF, aBD, obs, geoF, bdft, loc = inputs

            # Get batch indices
            current_ind = torch.tensor(IND[batch_idx], device=device)

            # Forward pass - ADD IND_batch argument
            outputs = model(aLS, aLF, aBD, obs, geoF, bdft, loc, current_ind)

            # Unpack outputs
            z_LS, z_LF, z_BD = outputs[:3]

            # Store predictions
            test_tensors['PLS'][current_ind] = z_LS.view(-1, 1)
            test_tensors['PLF'][current_ind] = z_LF.view(-1, 1)
            test_tensors['PBD'][current_ind] = z_BD.view(-1, 1)

            # Store causal parameters
            causal_params = {
                'ALS': model.z_ALS_K.detach() if hasattr(model, 'z_ALS_K') else None,
                'ALF': model.z_ALF_K.detach() if hasattr(model, 'z_ALF_K') else None,
                'ABD': model.z_ABD_K.detach() if hasattr(model, 'z_ABD_K') else None,
                'GammaLS': model.z_GammaLS_K.detach() if hasattr(model, 'z_GammaLS_K') else None,
                'GammaLF': model.z_GammaLF_K.detach() if hasattr(model, 'z_GammaLF_K') else None,
                'LambdaLS': model.z_LambdaLS_K.detach() if hasattr(model, 'z_LambdaLS_K') else None,
                'LambdaLF': model.z_LambdaLF_K.detach() if hasattr(model, 'z_LambdaLF_K') else None,
                'LambdaBD': model.z_LambdaBD_K.detach() if hasattr(model, 'z_LambdaBD_K') else None
            }

            # Update test tensors with causal parameters
            for key, param in causal_params.items():
                if param is not None:
                    test_tensors[key][current_ind] = param.view(-1, 1)

            # Only need to capture global parameters once
            if batch_idx == 0:
                # Store global model parameters
                global_params['w_0y'] = model.w_0y.detach().cpu()
                global_params['w_ey'] = model.w_ey.detach().cpu()
                global_params['w_0BD'] = model.w_0BD.detach().cpu()
                global_params['w_0LS'] = model.w_0LS.detach().cpu()
                global_params['w_0LF'] = model.w_0LF.detach().cpu()
                global_params['w_eLS'] = model.w_eLS.detach().cpu()
                global_params['w_eLF'] = model.w_eLF.detach().cpu()
                global_params['w_eBD'] = model.w_eBD.detach().cpu()

                # If you have additional parameters, add them here
                if hasattr(model, 'w_aLS'):
                    global_params['w_aLS'] = model.w_aLS.detach().cpu()
                if hasattr(model, 'w_aLF'):
                    global_params['w_aLF'] = model.w_aLF.detach().cpu()
                if hasattr(model, 'w_aBD'):
                    global_params['w_aBD'] = model.w_aBD.detach().cpu()

                # Store kernel parameters
                for param_name in ['ls_GammaLS', 'ls_GammaLF', 'ls_LambdaLS',
                                   'ls_LambdaLF', 'ls_LambdaBD', 'ls_ALS',
                                   'ls_ALF', 'ls_ABD', 'var_GammaLS',
                                   'var_GammaLF', 'var_LambdaLS', 'var_LambdaLF',
                                   'var_LambdaBD', 'var_ALS', 'var_ALF', 'var_ABD']:
                    if hasattr(model, param_name):
                        global_params[param_name] = getattr(model, param_name).detach().cpu()

    print("Evaluation complete")

    # Convert spatial tensors to CPU
    spatial_tensors = {k: v.cpu() for k, v in test_tensors.items()}

    # Return all tensors and parameters
    return spatial_tensors, global_params

import torch
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from contextlib import nullcontext


def prepare_data(config, PLS, PLF, PBD, GeoFeatures, DPM, BF, LOCAL):
    """
    Prepare data for the spatial model with improved handling of
    spatial coherence across mini-batches.

    Args:
        config: Configuration object
        PLS, PLF, PBD: Landslide, liquefaction, and building damage priors
        GeoFeatures: Geospatial features
        DPM: Damage proxy map observations
        BF: Building footprint tensor
        LOCAL: Local model tensor

    Returns:
        DataLoader, test tensors, and batch indices
    """
    # Get dimensions
    height, width = PLS.size(0), PLS.size(1)
    total_num = height * width

    # Calculate number of batches
    bnum = int(np.floor(total_num / config.batch_size))

    # Generate random indices for batch sampling while preserving spatial coherence
    # This step is critical for maintaining the global covariance structure
    rand_idx = np.array(random.sample(range(int(bnum) * config.batch_size), int(bnum) * config.batch_size))
    IND = torch.from_numpy(rand_idx[0:int(bnum * config.batch_size)])
    IND = IND.view(-1, config.batch_size)
    IND = IND.numpy()

    # Get device for computation
    device = config.device

    # Make sure GeoFeatures is on CPU for processing
    GeoFeatures_cpu = GeoFeatures.to('cpu')

    # Process data tensors
    # Create numpy arrays for all data components
    aLS = np.array(PLS.cpu()).reshape([total_num])[rand_idx[0:int(bnum * config.batch_size)]]
    aLF = np.array(PLF.cpu()).reshape([total_num])[rand_idx[0:int(bnum * config.batch_size)]]
    aBD = np.array(PBD.cpu()).reshape([total_num])[rand_idx[0:int(bnum * config.batch_size)]]
    obs = np.array(DPM.cpu()).reshape([total_num])[rand_idx[0:int(bnum * config.batch_size)]]
    geo = np.array(GeoFeatures_cpu).reshape([total_num, config.geofeature_dim])[rand_idx[0:int(bnum * config.batch_size)]]
    ft = np.array(BF.cpu()).reshape([total_num])[rand_idx[0:int(bnum * config.batch_size)]]
    loc = np.array(LOCAL.cpu()).reshape([total_num])[rand_idx[0:int(bnum * config.batch_size)]]

    # Convert to torch tensors and move to device
    LS = torch.from_numpy(aLS).float().view(-1, config.batch_size).to(device) + 1e-8
    LF = torch.from_numpy(aLF).float().view(-1, config.batch_size).to(device) + 1e-8
    BD = torch.from_numpy(aBD).float().view(-1, config.batch_size).to(device) + 1e-8
    OB = torch.from_numpy(obs).float().view(-1, config.batch_size).to(device) + 1e-8
    GeoF = torch.from_numpy(geo).float().view(-1, config.batch_size, config.geofeature_dim).to(device) + 1e-8
    FT = torch.from_numpy(ft).float().view(-1, config.batch_size).to(device)
    LOC = torch.from_numpy(loc).float().view(-1, config.batch_size).to(device)

    # Create test tensors on the target device
    test_PLS = PLS.reshape(total_num, 1).to(device)
    test_PLF = PLF.reshape(total_num, 1).to(device)
    test_PBD = PBD.reshape(total_num, 1).to(device)

    # Create dataset and dataloader
    dataset = tr_Dataset(LS, LF, BD, OB, GeoF, FT, LOC)
    MyDataLoader = DataLoader(dataset=dataset, batch_size=1, shuffle=False)

    return MyDataLoader, test_PLS, test_PLF, test_PBD, IND

total_num = PLS.size(0) * PLS.size(1)
config = Config()

# Add mixed precision setup here
if config.use_mixed_precision and torch.cuda.is_available():
    from torch.cuda.amp import autocast, GradScaler
    scaler = torch.amp.GradScaler('cuda')  # Updated GradScaler syntax
else:
    from contextlib import nullcontext
    autocast = lambda: nullcontext()
    class DummyScaler:
        def scale(self, loss): return loss
        def unscale_(self, optimizer): pass
        def step(self, optimizer): optimizer.step()
        def update(self): pass
    scaler = DummyScaler()

height, width = PLS.size(0), PLS.size(1)
total_num = PLS.size(0) * PLS.size(1)

# Make sure all input tensors are on the same device before model creation
device = config.device
PLS = PLS.to(device)
PLF = PLF.to(device)
PBD = PBD.to(device)
GeoFeatures = GeoFeatures.to(device)
DPM = DPM.to(device)
BF = BF.to(device)
LOCAL = LOCAL.to(device)

# Prepare data using the new function
MyDataLoader, test_PLS, test_PLF, test_PBD, IND = prepare_data(
    config, PLS, PLF, PBD, GeoFeatures, DPM, BF, LOCAL
)

# Initialize model
total_num = PLS.size(0) * PLS.size(1)
model = SpatialModel(config, total_num, sigma, GeoFeatures, PLS=PLS, PLF=PLF, PBD=PBD)

# Move entire model to the target device
if config.cuda:
    print("Model on GPU")
    model = model.to(config.device)

# Set optimizer
# opt = optim.RMSprop(model.parameters(), lr=config.learning_rate, momentum=0.9)
opt = optim.Adam(model.parameters(), lr=config.learning_rate)

test_results, global_params = run(MyDataLoader, model, opt, test_PLS, test_PLF, test_PBD, IND, config)

# Unpack spatial results
(test_PLS, test_PLF, test_PBD, test_ALS, test_ALF, test_ABD,
 test_GammaLS, test_GammaLF, test_LambdaLS, test_LambdaLF, test_LambdaBD) = test_results

# Now you can also access global parameters
w_0y = global_params['w_0y']
w_ey = global_params['w_ey']
w_0BD = global_params['w_0BD']
w_0LS = global_params['w_0LS']
w_0LF = global_params['w_0LF']
w_eLS = global_params['w_eLS']
w_eLF = global_params['w_eLF']
w_eBD = global_params['w_eBD']

Running k-means clustering on 10000 points to select 100 inducing points...
Selected inducing points with shape: torch.Size([100, 6])
Model on GPU
Training progress:


Epoch 1/20:   0%|          | 0/17596 [00:00<?, ?it/s]

Epoch 1/20 completed:
  Total Loss: 2.3143
  data_fit: -0.3591
  kl_inducing: 1948.1381
  reg_loss: 0.0008
  total_loss: 2.3143


Epoch 2/20:   0%|          | 0/17596 [00:00<?, ?it/s]

Epoch 2/20 completed:
  Total Loss: 0.1509
  data_fit: -0.1443
  kl_inducing: 0.0800
  reg_loss: 0.0008
  total_loss: 0.1509


...

Epoch 19/20 completed:
  Total Loss: 0.1838
  data_fit: -0.1819
  kl_inducing: 0.0800
  reg_loss: 0.0008
  total_loss: 0.1838


Epoch 20/20:   0%|          | 0/17596 [00:00<?, ?it/s]

Epoch 20/20 completed:
  Total Loss: 0.1747
  data_fit: -0.1728
  kl_inducing: 0.0800
  reg_loss: 0.0008
  total_loss: 0.1747
Evaluating model on all data...


Evaluating:   0%|          | 0/17596 [00:00<?, ?it/s]

Evaluation complete


# Evaluation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
original_shape = (DPM.shape[0], DPM.shape[1])
test_PLF1 = test_PLF.detach().cpu().numpy()
test_PBD1 = test_PBD.detach().cpu().numpy()
test_PLS1 = test_PLS.detach().cpu().numpy()
test_ALS = test_ALS.cpu().detach().numpy()
test_ALF = test_ALF.cpu().detach().numpy()
test_ABD = test_ABD.cpu().detach().numpy()
test_GammaLS1 = test_GammaLS.cpu().detach().numpy()
test_GammaLF1 = test_GammaLF.cpu().detach().numpy()
test_LambdaLS1 = test_LambdaLS.cpu().detach().numpy()
test_LambdaLF1 = test_LambdaLF.cpu().detach().numpy()
test_LambdaBD1 = test_LambdaBD.cpu().detach().numpy()

LOCAL1 = LOCAL.cpu().detach().numpy()
LOCAL_1_3_5_6_mask = ((LOCAL1 == 1) | (LOCAL1 == 3) | (LOCAL1 == 5) | (LOCAL1 == 6))
test_GammaLS1_reshape = test_GammaLS1.reshape(original_shape)
new_test_GammaLS1 = LOCAL_1_3_5_6_mask*test_GammaLS1_reshape

LOCAL_2_4_5_6_mask = ((LOCAL1 == 2) | (LOCAL1 == 4) | (LOCAL1 == 5) | (LOCAL1 == 6))
test_GammaLF1_reshape = test_GammaLF1.reshape(original_shape)
new_test_GammaLF1 = LOCAL_2_4_5_6_mask*test_GammaLF1_reshape

LOCAL_3_4_6_mask = ((LOCAL1 == 3) | (LOCAL1 == 4) | (LOCAL1 == 6))
test_GammaLS_reshaped = new_test_GammaLS1
test_GammaLF_reshaped = new_test_GammaLF1

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

im1 = axes[0].imshow(test_GammaLS_reshaped, cmap='Reds')
axes[0].set_title('GammaLS')
fig.colorbar(im1, ax=axes[0])
im2 = axes[1].imshow(test_GammaLF_reshaped, cmap='Blues')
axes[1].set_title('GammaLF')
fig.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()